 <h1 align="center">EDSA MOVIE RECOMMENDATION CHALLENGE 2021</h1> 

 <h3 align="center">Team ZM5</h3> 


<div align="center" style="width: 900px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="./streamlit_app/resources/imgs/Company_logo-alt.png"
     alt="Barnicles on your ship :( "
     style="float: center; padding-bottom=0.5em"
     width=900px/>

</div>

 <h2 align="left">Table Of Content</h2> 

<a id="TOC"></a>
- [1) Introduction](#1)
  * [1.1) Problem Statement](#2)
  * [1.2)  Objectives](#3)
  * [1.3) Data Overview](#3)
  * [1.4) Installing packages](#3)
- [2) Initializing Comet ](#4)
- [3) Importing Libraries ](#5)
- [4) Loading dataset](#6)
- [5) Exploratory Data Analysis (EDA)](#7)
  * [5.1) Detailed Dataset Overview](#8)
    * [5.1.1) Movie dataframe Overview](#9)
    * [5.1.2) IMdb dataframe Overview](#10)
    * [5.1.3) Genome Scores dataframe Overview](#11)
    * [5.1.4) Genome Tags dataframe Overview](#12)
    * [5.1.5) Tags dataframe Overview](#13)
    * [5.1.6) Links dataframe Overview](#14)
    * [5.1.7) train dataframe Overview](#15)
  * [5.2) Data visualisation](#16)
    * [5.2.1) Popular Movie Genre](#17)
    * [5.2.2) Popular Actors](#18)
    * [5.2.3) Popular Movie Directors](#19)
    * [5.2.4) Popular Play Plot](#20)
    * [5.2.5) Number of Movies Produced annually](#21)
    * [5.2.6) Movies tags plot annually](#22)
    * [5.2.7) Merging Movies and IMdb Dataframe](#23)
    * [5.2.8) Movies Budget Anually ](#24)
    * [5.2.9) User Movie rating plots ](#25)
- [6) Data Preprocessing](#26)
- [7) Modelling](#27)
  * [7.1) Content-based Recommendation](#28)
  * [7.2) Collaborative Filtering ](#29)
- [8) Hyperparameter Tuning](#31)
- [9) Creating a Kaggle Submission](#34) 
- [10) Model Pickling](#35)
- [11) Conclusion](#36)


# 1 ) Introduction

##  1.1 ) Problem Statement

In today’s technology driven world, recommender systems are socially and economically critical for ensuring that individuals can make appropriate choices surrounding the content they engage with on a daily basis. One application where this is especially true surrounds movie content recommendations; where intelligent algorithms can help viewers find great titles from tens of thousands of options.

The task then becomes to construct a recommendation algorithm based on content or collaborative filtering, capable of accurately predicting how a user will rate a movie they have not yet viewed based on their historical preferences.

Providing an accurate and robust solution has immense economic potential, with users of the system being exposed to content they would like to view or purchase - generating revenue and platform affinity.

## 1.2 ) Project Objectives

- To develop the best /effecient movie recommend.
- To compare contest-based and collaborating filtering in.

- To obtain an optimum unsupervised machine learning model with  RSME lower than 0.85 on the complete test set.

##  1.3 ) Data Overview

This dataset consists of several million 5-star ratings obtained from users of the online MovieLens movie recommendation service. The MovieLens dataset has long been used by industry and academic researchers to improve the performance of explicitly-based recommender systems, and now you get to as well!

For this Predict, we'll be using a special version of the MovieLens dataset which has enriched with additional data, and resampled for fair evaluation purposes.

## 1.4) Installing packages
Please download all required packages.

In [14]:
#!pip install LibRecommender
#!pip install surprise
#!pip install comet_ml
#!pip install seaborn
#!pip install wordcloud
!pip install libreco

ERROR: Could not find a version that satisfies the requirement libreco
ERROR: No matching distribution found for libreco


## 2) Initializing Comet.ml

Comet.ml is an experiment tracking library, useful for logging machine learning model development.

In [11]:
# Always import comet_ml at the top, before importing other packages
import comet_ml
from comet_ml import Experiment

In [12]:
# Create an experiment with your api key for version control
experiment = Experiment(
    api_key="amGQj1TC3Wyk9LYf7bD9xNV6G",
    project_name="movie-reccomender-edsa",
    workspace="daniel-bru",
)

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/daniel-bru/movie-reccomender-edsa/7f6b5b933897404e8ef3613fbe15feab
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (10 MB)
COMET INFO:     installed packages       : 1
COMET INFO: ---------------------------
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/daniel-bru/movie-reccomender-edsa/9e2d2177b57a43fca67ca083b1dca921



## 3) Importing Libraries

In [13]:
# Import packages here
# Packages for data processing
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
import re
from scipy.sparse import csr_matrix
import scipy as sp
import random

# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from re import sub
from decimal import Decimal
import locale
#locale.setlocale(locale.LC_ALL, 'en_US.UTF8')
sns.set(font_scale=1.4)
sns.set_style("darkgrid")
%matplotlib inline

# Packages for modeling
from surprise import Reader
from surprise import Dataset
from surprise import KNNWithMeans
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import SVD, BaselineOnly
from surprise import SVDpp as SSVDpp
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
import operator
import heapq

from libreco.data import random_split, DatasetPure, DataInfo
from libreco.data import split_by_ratio_chrono, DatasetFeat
from libreco.algorithms import SVD as libSVD
from libreco.algorithms import SVDpp, BPR
from libreco.algorithms import FM, WideDeep, DeepFM, AutoInt, DIN
from libreco.evaluation import evaluate
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_WARNINGS"] = "FALSE"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
def reset_state(name):
    tf.compat.v1.reset_default_graph()
    print("\n", "=" * 30, name, "=" * 30)

# Packages for model evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from time import time
from time import sleep
import gc

# Package to suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Packages for saving models
import pickle

ModuleNotFoundError: No module named 'libreco'

# 4) Loading Data data

In [ ]:
root_path = ''
df_sample_submission = pd.read_csv(root_path + 'sample_submission.csv')
df_movies = pd.read_csv(root_path + 'movies.csv')
df_imdb = pd.read_csv(root_path + 'imdb_data.csv')
df_genome_scores = pd.read_csv(root_path + 'genome_scores.csv')
df_genome_tags = pd.read_csv(root_path + 'genome_tags.csv')
train = pd.read_csv(root_path + 'train.csv')
df_test = pd.read_csv(root_path + 'test.csv')
df_tags = pd.read_csv(root_path + 'tags.csv')
df_links = pd.read_csv(root_path + 'links.csv')

 ## 5) Exploratory Data Analysis

### 5.1) Detailed Dataset Overview

### 5.1.1) Movie Dataframe Overview

In [ ]:
pd.set_option('display.max_colwidth', 600)
df_movies.head()

#### Movie dataframe information

In [ ]:
df_movies.info()

In [ ]:
df_movies.isnull().sum()

### Movie dataframe insight

 Movie dataframe has :
 * Three  features 1) movieId ,2) title  and 3) genres
 * The dataframe has 62423 observations
 * The dataframe has no cell with missing data
 * The dataframe has a mamory usage of 1.4MB

### 5.1.2) IMdb Dataframe Overview

In [ ]:
df_imdb.head()

In [ ]:
df_imdb.info()

In [ ]:
df_imdb.isnull().sum()

### IMdb dataframe insight

 IMdb dataframe has :
 * Six  features  namely  movieId, title_cast, director, runtime, budget and play_keywords
 * The dataframe has 27278 cells of observations 
 * The title_cast feature has 17210 observations and 10068 missing observations
 * The director feature has 17404 observations and  9874 missing observations
 * The runtime feature has 15189 observations and 12089 missing observations
 * The budget feature has 7906 observations and  19372 missing observations
 * The plot_keywords feature has 16200 observations and  11078 missing observations
 * The dataframe has a memory usage of 1.2MB

### 5.1.2) Genome Scores Dataframe Overview

In [ ]:
df_genome_scores.head()

In [ ]:
df_genome_scores.info()

In [ ]:
df_genome_scores.isnull().sum()

### Genome Scores dataframe insight

Genome Scores  dataframe has :
 * Three  features 1) movieId ,2) tagId  and 3) relevance
 * The dataframe has 15584448  observations
 * The dataframe has no cell with missing data
 * The dataframe has a mamory usage of 356.7MB

### 5.1.3) Genome Tags Dataframe Overview

In [ ]:
df_genome_tags.head()

In [ ]:
df_genome_tags.info()

In [ ]:
df_genome_tags.isnull().sum()

### Genome Tags dataframe insight

Genome Scores  dataframe has :
 * Three  features 1) tagId and 2) tag
 * The dataframe has 1128  observations
 * The dataframe has no cell with missing data
 * The dataframe has a mamory usage of 17.8KB

### 5.1.4) Tags Dataframe Overview

In [ ]:
df_tags.head()

In [ ]:
df_tags.info()

In [ ]:
df_tags.isnull().sum()

### Tags dataframe insight

Tags dataframe has :
 * Four  features 1) userId, 2) movieId , 3) tag and 4) timestamp 
 * The dataframe has  1093360  observations
 * The tag feature has 16 missing observations
 * The dataframe has a mamory usage of 33.5MB

### 5.1.5) Link Dataframe Overview

In [ ]:
df_links.head()

In [ ]:
df_links.info()

In [ ]:
df_links.isnull().sum()

### Links dataframe insight

Genome Scores  dataframe has :
 * Three  features 1) movieId , 2) imdbId and 3) tmbId
 * The dataframe has 62423 observations
 * The tmdbIdfeature has 107 messing observations
 * The dataframe has a mamory usage of 1.4MB

### 5.1.6) Train Dataframe Overview

In [ ]:
train.head() 

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

### Train dataframe insight

Train  dataframe has :
 * Four  features 1) userIdId , 2) movieId , 3) rating and 4) timestamp
 * The dataframe has 10000038   observations
 * The dataframe has no cell with missing data
 * The dataframe has a mamory usage of 305.2MB

## 5.2 ) Data Visualisation

### 5.2.1) Popular  Movie Genre Plot

In [ ]:
# Create dataframe containing only the movieId and genres
movies_genres = pd.DataFrame(df_movies[['movieId', 'genres']],columns=['movieId', 'genres'])

# Split genres seperated by "|" and create a list containing the genres allocated to each movie
movies_genres.genres = movies_genres.genres.apply(lambda x: x.split('|'))

# Create expanded dataframe where each movie-genre combination is in a seperate row
movies_genres = pd.DataFrame([(tup.movieId, d) for tup in movies_genres.itertuples() for d in tup.genres],
                             columns=['movieId', 'genres'])

In [ ]:
# Plot the genres from most common to 90 common
plot = plt.figure(figsize=(14, 8), dpi =95)
plt.title('Popular Movie Genres\n', fontsize=20)
sns.countplot(y="genres", data=movies_genres,order=movies_genres['genres']
              .value_counts(ascending=False).index, color='b', ec= 'red', alpha=0.9)
plt.ylabel('')
plt.xlabel('Count')
plt.show()

### 5.2.2) Popular Actors  Plot

In [ ]:
movies_actor = pd.DataFrame(df_imdb[['movieId', 'title_cast']],columns=['movieId', 'title_cast'])

# Split title_cast seperated by "|" and create a list containing the title_cast allocated to each movie
movies_actor= movies_actor[movies_actor['title_cast'].notnull()]
movies_actor.title_cast = movies_actor.title_cast.apply(lambda x: x.split('|'))

# Create expanded dataframe where each movie-tite_cast combination is in a seperate row
movies_actor = pd.DataFrame([(tup.movieId, d) for tup in movies_actor.itertuples() for d in tup.title_cast],columns=['movieId','title_cast'])

movies_actor = movies_actor.groupby(['title_cast'])['movieId'].count().reset_index(name='Number of Movies')
movies_actor =movies_actor.sort_values(by='Number of Movies',ascending=False)

#Sececting the Top 20 actors in movies
movies_actor = movies_actor .head(20)
movies_actor =movies_actor.sort_values(by='Number of Movies',ascending=True)


In [ ]:
y_labels =movies_actor['title_cast']

# Plot the figure.
plt.figure(figsize=(17, 12),dpi =85)
ax = movies_actor['Number of Movies'].plot(kind='barh',color='b', fontsize=17,
                                           edgecolor='red', xlim=[45,84], width=.75, alpha=0.8)
ax.set_title('Top 20 Actors Featured in the IMDB Database', fontsize=20)
ax.set_xlabel('Number of movies featuring the actor')
ax.set_ylabel('')
ax.set_yticklabels(y_labels)
rects = ax.patches

for rect in rects:
    # Get X and Y placement of label from rect.
    x_value = rect.get_width()
    y_value = rect.get_y() + rect.get_height() / 2
    # Number of points between bar and label. Change to your liking.
    space = 5
    # Vertical alignment for positive values
    ha = 'left'
    if x_value < 0:
        space *= -1
        ha = 'right'
    
    label = "{}".format(x_value)
    plt.annotate(label,(x_value, y_value),xytext=(space, 0),fontsize=16,
                 textcoords="offset points",va='center',ha=ha)            
                                 
        
    

### 5.2.3) Popular Movie directors 

In [ ]:

## grouping the movies by the director and counting the total number of movies per director
movies_director = pd.DataFrame(df_imdb[['movieId', 'director']],columns=['movieId', 'director'])

movies_director  = movies_director.groupby(['director'])['movieId'].count().reset_index(name="count")
movies_director =movies_director.sort_values(by='count',ascending=False)

movies_director = movies_director .head(20)
movies_director =movies_director.sort_values(by='count',ascending=True)

In [ ]:
y_labels =movies_director['director']

# Plot the figure.
plt.figure(figsize=(18, 12), dpi =85)
ax = movies_director['count'].plot(kind='barh',color='b', edgecolor='red', 
                                   width=.7, fontsize=16, xlim=[8,30], alpha=0.9)
ax.set_title('Top 20 Directors Featured in the IMDB Database', fontsize=18)
ax.set_xlabel('Number of movies directed')
ax.set_ylabel('')
ax.set_yticklabels(y_labels)

rects = ax.patches

for rect in rects:
    # Get X and Y placement of label from rect.
    x_value = rect.get_width()
    y_value = rect.get_y() + rect.get_height() / 2
    # Number of points between bar and label. Change to your liking.
    space = 5
    # Vertical alignment for positive values
    ha = 'left'

    if x_value < 0:
        space *= -1
        ha = 'right'
    label = "{}".format(x_value)

    # Create annotation
    plt.annotate(label,(x_value, y_value),xytext=(space, 0),textcoords="offset points"
                 ,va='center',ha=ha, fontsize=16)                                        


### 5.2.4) Popular Play Plots 

In [ ]:
movies_plot = pd.DataFrame(df_imdb[['movieId', 'plot_keywords']],
                             columns=['movieId', 'plot_keywords'])

# Split play plot seperated by "|" and create a list containing the play plot allocated to each movie
movies_plot= movies_plot[movies_plot['plot_keywords'].notnull()]
movies_plot.plot_keywords = movies_plot.plot_keywords.apply(lambda x: x.split('|'))

# Create expanded dataframe where each movie-play_plot combination is in a seperate row
movies_plot = pd.DataFrame([(tup.movieId, d) for tup in movies_plot.itertuples() for d in tup.plot_keywords],columns=['movieId','plot_keywords'])

movies_plot = movies_plot.groupby(['plot_keywords'])['movieId'].count().reset_index(name="count")
movies_plot =movies_plot.sort_values(by='count',ascending=False)

movies_plot = movies_plot.head(20)
movies_plot =movies_plot.sort_values(by='count',ascending=True)

In [ ]:
y_labels =movies_plot['plot_keywords']

# Plot the figure.
plt.figure(figsize=(18, 12), dpi=85)
ax = movies_plot ['count'].plot(kind='barh',color='darkblue', fontsize=17,
                                           edgecolor='r', width=.7, alpha=0.7)
ax.set_title('Top 20 Popular Movie Plots', fontsize=19)
ax.set_xlabel('Total Number of Movie Plots')
ax.set_ylabel('Plot')
ax.set_yticklabels(y_labels)

rects = ax.patches

for rect in rects:
    # Get X and Y placement of label from rect.
    x_value = rect.get_width()
    y_value = rect.get_y() + rect.get_height() / 2
    # Number of points between bar and label. Change to your liking.
    space = 5
    # Vertical alignment for positive values
    ha = 'left'

    if x_value < 0:
        
        space *= -1

        ha = 'right'

    
    label = "{}".format(x_value)
    
     # Create annotation
    plt.annotate(label,(x_value, y_value),xytext=(space, 0),
                 textcoords="offset points",va='center',ha=ha, fontsize=17) 


### 5.2.5)  Number  of Movies Produced Annually

In [ ]:
# Create dataframe containing only the movieId and genres
movies_year = pd.DataFrame(df_movies[['movieId', 'title']],columns=['movieId', 'title'])

movies_year= movies_year[movies_year['title'].notnull()]
# Split the year from title
movies_year.title = movies_year.title.apply(lambda x: x[-5:-1])

movies_year = movies_year.groupby(['title'])['movieId'].count().reset_index(name="count")
movies_year =movies_year[movies_year['count'] > 500]
movies_year =movies_year.sort_values(by='count',ascending=True)


In [ ]:
y_labels =movies_year['count']

# Plot the figure.
fig,ax = plt.subplots(figsize=(18, 10), dpi = 85)

movies_year[['count', 'title']].plot(x='title',kind='bar', color='white', fontsize=17, ylim=[100, 2600],
                                           width=.9, ax=ax, legend=False, edgecolor='darkblue')
                                     
movies_year.plot(x='title', kind = 'line', ax=ax, color='purple', legend=False)
                                     
ax.set_title('Number of Movies Produced Annually', fontsize=22)
ax.set_xlabel('Year')
ax.set_ylabel('Total Number of Movies Produced', fontsize=19)
                                     
plt.xticks(rotation=70)
plt.show()

### 5.2.6)  Movie tags

In [ ]:
tags_2 =str(list(df_tags['tag']))
plt.figure(figsize = (20,20))
wc = WordCloud(background_color = "black", max_words = 100 , width = 1600 , height = 800,
               collocations=False).generate(tags_2)
plt.axis("off")
plt.imshow(wc)

### 5.2.6)  Merging Movies and IMdb Dataframe

In [ ]:
merged_inner = pd.merge(left=df_movies, right=df_imdb, left_on='movieId', right_on='movieId')
merged_inner.head(2)

### 5.2.7)  Movie's Annual Budget 

In [ ]:
movies_ =df_imdb.sort_values(by='budget',ascending=False)
print(movies_.budget.unique())

The bugdet feature has different currency.In this notebook we will consider US dollars \\$ ,because most movies are in \\$ 

In [ ]:
movies_budget = pd.DataFrame(merged_inner[['movieId', 'title','budget']],columns=['movieId', 'title','budget'])

movies_budget= movies_budget[movies_budget['title'].notnull()]

movies_budget['budget'].replace('', np.nan, inplace=True)
movies_budget.dropna(subset=['budget'], inplace=True)

# Split title seperated by "|" and create a list containing the title allocated to each movie
movies_budget.title = movies_budget.title.apply(lambda x: int(re.findall(r"\d\d\d\d",x).pop(0)))
movies_budget.budget = movies_budget.budget.apply(lambda x: int(re.sub("[^0-9]", "", x)) if str(x)[:1] =='$' else None)
movies_budget = movies_budget[movies_budget['budget'].notnull()]

The list of years and  the box and whisker shows that there was a movie released in the year 1000 , this cannot be true.The first basic movie was made in 1888. Also the data has movies made beyod 2021 (such as a movie made in 3000) this  values are cearly outliers.

In [ ]:
print('list of movie\'s year of release :', list(pd.unique(movies_budget['title'])))

In [ ]:
fig,ax = plt.subplots(figsize=(25, 2), dpi=65)
sns.boxplot(x='title', data=movies_budget, palette='hot')
ax.set_xlabel('Year')
plt.show()

We then select movies that are made from 1888 up until the current year 2021

In [ ]:
movies_budget =movies_budget[movies_budget['title'] > 1888]
movies_budget =movies_budget[movies_budget['title'] <= 2021]

In [ ]:
fig,ax = plt.subplots(figsize=(25, 2), dpi=65)
sns.boxplot(x = 'title', data = movies_budget, color='b')
ax.set_xlabel('Year')
plt.show()

The above box and whisker is skewed to the right,this is an indication that more movies where released from the late 90s.This is because of technogyical advances and more investment/budget to the production of movies.

In [ ]:
movies_budget =movies_budget[movies_budget['title'] > 1995]
movies_budget =movies_budget[movies_budget['title'] <= 2012]
Total_movie_budget = movies_budget.groupby("title")["budget"].sum().reset_index(name="Sum of movie budget")
Total_movie_budget

In [ ]:
fig,ax = plt.subplots(figsize=(17, 8), dpi=85)

Total_movie_budget.plot.line(x='title',y='Sum of movie budget', ax=ax, color='blue', 
                             legend=False, fontsize=16)
ax.set_title('Sum of Total Movie budget per year')
ax.set_xlabel('Year')
ax.set_ylabel('Sum of movie budget   (billion)')

plt.show()

### 5.2.8)  User Movie rating plots

Merging movie and train

In [ ]:
df_train = train.copy()
merged_train = pd.merge(left=df_movies, right=df_train, left_on='movieId', right_on='movieId')
merged_train.head(2)

In [ ]:
train_ = df_train.copy()
print("Total Number of Users who rated  movies:" ,len(train_['userId'].unique()))
print("Maximum movies rating :" ,merged_train['rating'].max())
print("Lowest movies rating :" ,merged_train['rating'].min())

In [ ]:
merged_train.rating = merged_train.rating.apply(lambda x: int(x))
Hi_rating_df =merged_train[merged_train['rating']==5]

Lo_rating_df =merged_train[merged_train['rating']==0]

Hi_rating_list =str(list(Hi_rating_df['title']))
Lo_rating_list =str(list(Lo_rating_df['title']))

### Movies that obtained the highest rating</h2>  

In [ ]:
plt.figure(figsize = (20,20))
wc = WordCloud(background_color = "black", max_words = 100 , 
               width = 1600 , height = 800,collocations=False).generate(Hi_rating_list)
plt.axis("off")
plt.imshow(wc)

### Movies that obtained the Lowest rating</h2>

In [ ]:
plt.figure(figsize = (20,20))
wc = WordCloud(background_color = "black", max_words = 100 , 
               width = 1600 , height = 800,collocations=False).generate(Lo_rating_list)
plt.axis("off")
plt.imshow(wc)

Some  movies such as the star wars movie, appears in the two word plot.This means its possible for a movie to recieve the highest and the lowest rating from different users.

In [ ]:
# Plot the ratings from most common to least common
ratings = train['rating'].value_counts(ascending=False).index.tolist()

plot = plt.figure(figsize=(12, 8))
plt.title('Most common ratings\n', fontsize=20)
sns.countplot(y="rating", data=train,
              order=ratings,
              palette='ocean', ec='r', alpha = 0.9)
plt.show()

Here we can see the distribution of the ratings. There seems to be quite a few movies with a prefect rating.
Movies with a perfect rating could be a result of it only being rated by a very small amount of people or even just once.

In [ ]:
temp_df = pd.merge(train, df_movies, on='movieId')

In [ ]:
# mean of ratings a movie has received
temp_df.groupby('title')['rating'].mean().head()

In [ ]:
temp_df.groupby('title')['rating'].mean().sort_values(ascending=False).head(10)

In [ ]:
# Movies which have been rated the most
temp_df.groupby('title')['rating'].count(
).sort_values(ascending=False).head(10)

Finally some movies I have seen.

In [ ]:
# creating df for ploting the mean for each movies ratings
rating_mean_count = pd.DataFrame(temp_df.groupby('title')['rating'].mean())
rating_mean_count['rating_count'] = pd.DataFrame(
    temp_df.groupby('title')['rating'].count())

In [ ]:
plt.rcParams['patch.force_edgecolor'] = True
sns.jointplot(x='rating', y='rating_count',
              data=rating_mean_count, alpha=0.8, height=12)
plt.title("Movies mean rating")

In [ ]:
# Count number of ratings movie has received
temp_df.groupby('title')['rating'].count().sort_values(ascending=True).head(10)

In [ ]:
rating_mean_count[rating_mean_count['rating_count'] == 1].count()

There is quite a few movies in this dataset which have only received one rating are then people niche tastes or are they hidden gems that the masses do not know about.

In [ ]:
# count of ratings a user left
temp_df.groupby('userId')['rating'].count(
).sort_values(ascending=True).head(10)

We can see that some users have only rated a very small amount of movies. We wont be able to to compare these users like or dislikes because there is too little information on them.

In [ ]:
# freeing up memory
gc.collect()
del Total_movie_budget, movies_budget, merged_inner
del plot, fig, wc, ax
del temp_df
del rating_mean_count

# 6) Data Preprocessing

### Recomendation Models

We will attempt to solve our problem by using:

* Collaborative model-based Filtering 

* Content-based filtering

* The Hybrid Approach

## Preprocessing

+ We create subset the train dataframe for users/movies above a certain threshold.
This subset is used in training some models

+ We create features we'll use later to pass to the LibRecommender hybrid models.


In [ ]:
df_train = train.copy()

"""Counting the number of times a movie has recieved an rating"""
rating_count = df_train.groupby('movieId')['rating'].count().reset_index()
rating_count.rename(columns={'rating': 'movie_rating_count'}, inplace=True)
df_train = df_train.merge(rating_count, on='movieId')

"""Counting the number of times a user has left an rating"""
user_count = df_train.groupby('userId')['rating'].count().reset_index()
user_count.rename(columns={'rating': 'user_rating_count'}, inplace=True)
df_train = df_train.merge(user_count, on='userId')

"""Droping all movies/users which are below a threshhold"""
mrc = 20
urc = 5
df_train = df_train[df_train['movie_rating_count'] > mrc]
df_train = df_train[df_train['user_rating_count'] > urc]
display(df_train.head(3))
print(df_train.shape)
df_train.drop(['movie_rating_count', 'user_rating_count'], 1, inplace=True)

display(df_train.head(3))
print(df_train.shape)

In [ ]:
def create_features(df, column, features):
    """Creates features from a column of lists where the list items will be the features."""

    features_df = pd.DataFrame(columns=features)
    all_features = pd.concat([df, features_df], axis=1)

    for i, row in all_features.iterrows():
        for feature in features:
            if feature in getattr(row, column):
                all_features[feature].iloc[i] = 1
    all_features = all_features.drop(column, axis=1)

    return all_features

In [ ]:
"""Fill NaN's and extract numbers from budget"""
df_imdb.fillna(0, inplace=True)
df_imdb.budget = df_imdb.budget.astype(str).apply(
    lambda x: int(re.sub("[^0-9]", "", x)))

In [ ]:
%%time
movies_genres = pd.DataFrame(df_movies[['movieId', 'genres']],
                             columns=['movieId', 'genres'])

# Split genres seperated by "|" and create a list containing the genres allocated to each movie
movies_genres.genres = movies_genres.genres.apply(lambda x: x.split('|'))
genres = ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
          'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
          'Mystery', 'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical',
          'Western', 'Film-Noir', '(no genres listed)']

genre_encode = create_features(movies_genres, 'genres', genres)
genre_encode.head(4)

In [ ]:
%%time
cast = pd.DataFrame(df_imdb[['movieId', 'title_cast']],
                    columns=['movieId', 'title_cast'])

# Split title_cast seperated by "|" and create a list containing the cast members in each movie
cast.title_cast = cast.title_cast.astype(str).apply(lambda x: x.split('|'))
cast_movie = pd.DataFrame([(tup.movieId, d) for tup in cast.itertuples() for d in tup.title_cast],
                          columns=['movieId', 'title_cast'])
mostc_cast = cast_movie['title_cast'].value_counts(
    ascending=False).index.tolist()[:16]

cast_encode = create_features(cast, 'title_cast', mostc_cast[1:])
cast_encode.head(4)

In [ ]:
gc.collect()
del rating_count, user_count, movies_genres, cast, cast_movie
sleep(2)

# 7) Modelling 


### 7.1) Content-based Recommendation

Given a movie or list of movies, we can look at its features and compare it to the features of others movies, in so doing,  we'll find similaries bewteen the movies and can reccomend movies based on the level of similarity it shares with the given movie or movie list. 

Here we implement a simple content-based recommender system, it will take a movie title and return a list of movies that are most similar to the title given. 

In [ ]:
movies_imdb = pd.merge(df_movies, df_imdb, on='movieId', how='left').fillna(0)

In [ ]:
movies_imdb.head(2)
print(movies_imdb.shape)

Here we have genres, cast, director and plot keywords which we will use as our 'content'.

In [ ]:
def join_cols(col1, col2, col3):
    """ return the content of individual columns as one """
    temp = []
    for i in range(len(col1)):
        temp.append(str(col1.iloc[i]) + '|' + str(col2.iloc[i]) +
                    '|' + str(col3.iloc[i]))
    return temp

In [ ]:
movies_imdb['joined'] = join_cols(
    movies_imdb.genres, movies_imdb.title_cast, movies_imdb.director)

In [ ]:
# droping the merged columns and columns we don't need
movies_imdb.drop(['genres', 'title_cast', 'director',
                  'plot_keywords', 'budget', 'runtime', 'plot_keywords'], 1, inplace=True)

In [ ]:
movies_imdb.head(2)

In [ ]:
# limiting
movies_imdb = movies_imdb[:20000]

In [ ]:
# this will be use as a custom tokenizer
def pipe_split(string):
    """ if string has a '|' split on it, if not just pass """
    if '|' in string:
        return string.split('|')
    else:
        return string

In [ ]:
# matrix's of word frequency scores
tf_vec = TfidfVectorizer(min_df=10, stop_words='english',
                         strip_accents='unicode', tokenizer=pipe_split, max_features=100)

tf = tf_vec.fit_transform(movies_imdb.joined)

In [ ]:
# compute the cosine similarity
cos_sim = linear_kernel(tf,tf)
cos_sim[0]

In [ ]:
# movie titles and index number
indices = pd.Series(movies_imdb.index,
                    index=movies_imdb['title']).drop_duplicates()

In [ ]:
indices.head()

In [ ]:
def get_rec(title, cos_sim=cos_sim):
    # get index of corresponding title
    idx = indices[title]
    
    # get the pairwise similarity scores
    sim_scores = list(enumerate(cos_sim[idx]))
    
    # sort movies
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]
    
    # movies indices
    movie_indices = [i[0] for i in sim_scores]
    
    # top 10 most similar movies
    return movies_imdb['title'].iloc[movie_indices]

The recommendations

In [ ]:
get_rec('Toy Story (1995)')

In [ ]:
gc.collect()
del cos_sim, movies_imdb, indices, tf
sleep(10)

### Using The Surprise Package

## 7.2) Collaborative Filtering 

In [ ]:
%%time
# we sample the datafrane for 1 mil rows
ratings = df_train.sample(1000000, random_state=1).copy()
ratings_dict = {'itemID': list(ratings.movieId),
                'userID': list(ratings.userId),
                'rating': list(ratings.rating)}
df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0.5, 5.0))
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

In [ ]:
%%time
trainset, testset = train_test_split(data, test_size=.01, random_state=1)

In [ ]:
gc.collect()
del ratings, ratings_dict, df, data
sleep(10)

#### SVD
SVD is a matrix factorisation technique, which reduces the number of features of a dataset by reducing the space dimension from N-dimension to K-dimension (where K<N).

In [ ]:
%%time
#experiment = Experiment(
#    api_key="amGQj1TC3Wyk9LYf7bD9xNV6G",
#    project_name="movie-reccomender-edsa",
#    workspace="daniel-bru",
#)

algo = SVD(verbose=True)
# Train the algorithm on the trainset, and predict ratings for the testset
start_time = time()
algo.fit(trainset)
predictions = algo.test(testset)
run_time = time() - start_time

# Compute Metrics
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

# Create and add metrics to Perfermance Dataframe
performance_df = pd.DataFrame()
ssvd_0 = {'model': 'SSVD', 'RMSE': rmse,
          'MAE': mae, 'execution_time': run_time}
performance_df = performance_df.append([ssvd_0], ignore_index=True)
display(performance_df)

# Log the parameters and results
##experiment.log_metrics(ssvd_0)
#experiment.set_code()
#experiment.end()

#### SVDpp

The SVD++ algorithm, an extension of SVD taking into account implicit ratings.

In [ ]:
%%time
#experiment = Experiment(
#    api_key="amGQj1TC3Wyk9LYf7bD9xNV6G",
#    project_name="movie-reccomender-edsa",
#    workspace="daniel-bru",
#)

algo = SSVDpp(random_state=1, verbose=True)

start_time = time()
algo.fit(trainset)
predictions = algo.test(testset)
run_time = time() - start_time

# Compute Metrics
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

# Add metrics to Perfermance Dataframe
ssvdpp_0 = {'model': 'SSVDpp', 'RMSE': rmse,
            'MAE': mae, 'execution_time': run_time}
performance_df = performance_df.append([ssvdpp_0], ignore_index=True)
display(performance_df)

# Log the parameters and results
#experiment.log_metrics(ssvdpp_0)
#experiment.set_code()
#experiment.end()

In [ ]:
# Assess Performance of Surprise Algorithms
display(performance_df)

# 8) Hyperparameter Tuning

#### SVD

In [ ]:
%%time
#experiment = Experiment(
#    api_key="amGQj1TC3Wyk9LYf7bD9xNV6G",
#    project_name="movie-reccomender-edsa",
#    workspace="daniel-bru",
#)

algo = SVD(random_state=1, n_factors=12, lr_all=0.007, reg_all=0.02, n_epochs=20,
           init_std_dev=0.005, init_mean=-0.025, lr_qi=0.01, lr_bi=0.005,
           verbose=True)
# Train the algorithm on the trainset, and predict ratings for the testset
start_time = time()
algo.fit(trainset)
predictions = algo.test(testset)
run_time = time() - start_time

# Compute Metrics
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

# Create and add metrics to Tuned Perfermance Dataframe
tuned_performance_df = pd.DataFrame()
ssvd_1 = {'model': 'SSVD_tuned', 'RMSE': rmse,
          'MAE': mae, 'execution_time': run_time}
tuned_performance_df = tuned_performance_df.append([ssvd_1], ignore_index=True)
display(tuned_performance_df)

# Log the parameters and results
#experiment.log_metrics(ssvd_1)
#experiment.set_code()
#experiment.end()

#### SVDpp

In [ ]:
%%time
#experiment = Experiment(
#    api_key="amGQj1TC3Wyk9LYf7bD9xNV6G",
#    project_name="movie-reccomender-edsa",
#    workspace="daniel-bru",
#)

algo = SSVDpp(random_state=1, n_factors=12, lr_all=0.007, reg_all=0.02, n_epochs=20,
              init_std_dev=0.005, init_mean=-0.025, verbose=True)
# Train the algorithm on the trainset, and predict ratings for the testset
start_time = time()
algo.fit(trainset)
predictions = algo.test(testset)
run_time = time() - start_time

# Compute Metrics
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

# Create and add metrics to Tuned Perfermance Dataframe
ssvdpp_1 = {'model': 'SSVDpp_tuned', 'RMSE': rmse,
            'MAE': mae, 'execution_time': run_time}
tuned_performance_df = tuned_performance_df.append(
    [ssvdpp_1], ignore_index=True)
tuned_performance_df.drop_duplicates(inplace=True)
display(tuned_performance_df)

# Log the parameters and results
#experiment.log_metrics(ssvdpp_1)
#experiment.set_code()
#experiment.end()

### Surprise Models Assessment

In [ ]:
tuned_performance_df = tuned_performance_df.set_index('model')
performance_df = performance_df.set_index('model')

In [ ]:
tuned_performance_df.plot(y='RMSE', kind='barh', xlim=[0.8, 0.92], color='blue', edgecolor='blue',
                          figsize=(15, 3), fontsize=10, legend=True, title='RMSE')

In [ ]:
tuned_performance_df.plot(y='execution_time', kind='barh', xlim=[20, 390], color='blue', edgecolor='blue',
                          figsize=(15, 3), fontsize=10, legend=True, title='Execution Time')
display(tuned_performance_df)
display(performance_df)

The execution time for surprise models plays a big role in deciding the best model. This is because increasing number of epochs doesn't always improve the models accuracy but always increases the execution time tremendously.

Due to the tremendously high exectution time of the surprise SVDpp and only holding a 0.0075 RMSE lead on the SVD, we wont be persuing it any longer. 

While these two models are the best performing from the surprise package, they might not be the best models across all the recommender algorithms packages. We will now shift our attention to other recommender packages and assess their algorithms performaces.

In [ ]:
gc.collect()
del algo, trainset, testset,
sleep(10)

### Using The LibRecommender Package

## Collaborative Filtering 

In [ ]:
%%time
# We are using the full train dataframe because we dont need to sample to save time.
data = train.copy()
data.rename(columns={'userId': 'user', 'movieId': 'item', 'rating': 'label',
                               'timestamp': 'time'}, inplace=True)

# split whole data into three folds for training, evaluating and testing
train_data, eval_data, test_data = random_split(
    data, multi_ratios=[0.99, 0.005, 0.005])

# building data sets
train_data, data_info = DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)
print(data_info)

In [ ]:
def reset_state(name):
    tf.compat.v1.reset_default_graph()
    print("\n", "=" * 30, name, "=" * 30)

#### SVD

In [ ]:
%%time
#experiment = Experiment(
#    api_key="amGQj1TC3Wyk9LYf7bD9xNV6G",
#    project_name="movie-reccomender-edsa",
#    workspace="daniel-bru",
#)

reset_state('libSVD')
svd_lib = libSVD(task="rating", data_info=data_info, embed_size=16, n_epochs=20, lr=0.004, batch_size=9000,
                 batch_sampling=True, reg=0.000001, num_neg=-5)  
# monitor metrics on eval_data during training
svd_lib.fit(train_data, verbose=2, eval_data=eval_data, metrics=["rmse", "mae", "r2"])

# do final evaluation on test data
metrics = evaluate(model=svd_lib, data=test_data, metrics=["rmse", "mae"])
print("evaluate_result: ", metrics)

# Log the parameters and results 
#experiment.log_metrics(metrics)
#experiment.set_code()
#experiment.end()

This model appears very quite fruit full. Being trained on the entire dataset, the exection time is 5 minutes on 20 epochs and gives us a very good RMSE of 0.8005 on the test set.

We can predict the rating a user might give a movie by calling the predict function.
We can also reccomend movies to a user by specifying that users userId and the number of recommendations into the libreccomender recommend_user function.

In [ ]:
prediction = svd_lib.predict(user=5163, item=57669)
recommendation = svd_lib.recommend_user(user=5163, n_rec=5)
print("prediction: ", prediction[0])

rec = []
ratings = []
recommended_movies = df_movies.copy()

for r in recommendation:
    rec.append(r[0])
    ratings.append(r[1])
    
recommended_movies = recommended_movies[recommended_movies['movieId'].isin(rec)].set_index('movieId').copy()
recommended_movies['rating'] = ratings

print('User 5163 will be recommended:')
display(recommended_movies.sort_values('rating', ascending =False))

In [ ]:
gc.collect()
del data, svd_lib
sleep(10)

#### SVDpp

In [ ]:
%%time
#experiment = Experiment(
#    api_key="amGQj1TC3Wyk9LYf7bD9xNV6G",
#    project_name="movie-reccomender-edsa",
#    workspace="daniel-bru",
#)

reset_state('SVDpp')
svdpp = SVDpp(task="rating", data_info=data_info, embed_size=16, n_epochs=2,
              lr=0.0048, batch_size=10000,
              batch_sampling=True, reg=0.000001, num_neg=-5)

svdpp.fit(train_data, verbose=2, eval_data=eval_data,
          metrics=["rmse", "mae", "r2"])

# do final evaluation on test data
metrics2= evaluate(model=svdpp, data=test_data, metrics=["rmse", "mae"])
print("evaluate_result: ", metrics2)


#experiment.log_metrics(metrics2)
#experiment.set_code()
#experiment.end()

The SVDpp from LibReccomender seems very robust, the more epochs we give it the better the test set RMSE. We have chosen 8 epochs in the end as RMSE drops to a nice 0.798 and the exection time per epoch is only 10min when running on google collab and 15min when run elsewhere.

In [ ]:
prediction = svdpp.predict(user=5163, item=57669)
recommendation = svdpp.recommend_user(user=5163, n_rec=5)
print("prediction: ", prediction[0])

rec = []
ratings = []
recommended_movies = df_movies.copy()

for r in recommendation:
    rec.append(r[0])
    ratings.append(r[1])
    
recommended_movies = recommended_movies[recommended_movies['movieId'].isin(rec)].set_index('movieId').copy()
recommended_movies['rating'] = ratings

print('User 5163 will be recommended:')
display(recommended_movies.sort_values('rating', ascending =False))

In [ ]:
gc.collect()
del svdpp, train_data, data_info, eval_data, test_data
sleep(10)

### LibRecommender Collaborative Model Assessment 

We cannot realy compare use the execution time for the comparing of the SVD and SVDpp here because the models are very robust and the more epochs we decide to give it, the better the model accuracy but greater the execution time. So depending on the situation, we could go for either.

Currently, from the number of epochs we ran, the LibReccomender SVDpp algorithm offers the best predictive ability. We will train the algorithm on the full training dataset and later use the model to predict the ratings for a test dataset containing, for the most part, familiar userId/movieId pairs. 

## Content-based Filtering
### and Hybrid Models

Currently we have a cold start problem that needs to be addressed. In our test data of user and movie ids, we have user-movie id pairs which are not in the training data, because the training data does not contain the movies that are linked with some of the users. The problem is that no one else has rated these movies before. As such, our collaborative models are limited in the manner in which they can predict how a user might rate these movies. These models cannot compare our adventurous users with others users who have rated these movies simply becuase we do not have data on it.

A simple way to address this would be to use Content-based Filtering.

In Content-based Filtering, we seek to make recommendations based on how similar the properties or features of an item are to other items. 
We can directly calculate what rating a user would give a movie that they haven't watched yet by comparing the properies of that movie with other movies they have watched. 

How the problem was found:
    
Our Librecommender collaborative algorithms are quite useful. When we use it to predict on our test data and it comes across a user-movie id combination it cannot use to compare users, it adds the index of the combination to a list and notifies us of it afterwards. 
We will be using this list and content-based filtering to try to make more accurate predictions for how the users might rate these unseen movies.

In [ ]:
# Indices of unknown user-movie id pairs from our test data
unknowns = [2228224, 2228225, 2228226, 1130497, 2605060, 2228229, 327686, 2228230, 2228231, 2228233, 327690, 2228234, 2605070, 2228239, 2228240, 2228241, 2605075, 2228244, 2228246, 2228249, 2228251, 2228253, 2228255, 2228256, 2228257, 360482, 2228258, 1425439, 2228261, 2605088, 2605089, 2228264, 376866, 606243, 1425445, 376870, 1425447, 2228270, 2228271, 2228272, 2228273, 2883631, 2359347, 2670636, 1916982, 2228280, 2228282, 2228284, 2670652, 2228286, 4898877, 2228288, 2228289, 2670655, 4767809, 2670665, 2228299, 2228301, 2228302, 2228303, 2883664, 2228305, 901200, 2228308, 2228309, 2687061, 2228311, 2883672, 901205, 2228314, 2228315, 2670682, 2228317, 2228318, 3342430, 2670688, 508002, 2228324, 2228325, 2228326, 508005, 508007, 508008, 4767850, 2228331, 508011, 4767851, 4767854, 508018, 2228339, 4079731, 4767858, 4767861, 4767862, 2228344, 2228346, 1245307, 2883707, 2228349, 4784250, 2228351, 2228352, 2670714, 2228354, 2228355, 2670717, 2228358, 2670726, 2228360, 2883720, 2883721, 2883722, 2228364, 2670729, 2359439, 2228368, 2883728, 2228370, 2228371, 2228372, 2228373, 508048, 2670741, 2228376, 2228377, 4931736, 2670746, 2883740, 2228381, 2228383, 2228385, 2228386, 475299, 2228389, 2359461, 2228391, 2670757, 2228395, 2228397, 2228398, 2883759, 2228400, 2359472, 2883760, 2228403, 2228404, 2883761, 2883764, 2883766, 3604663, 2228409, 4456631, 2228411, 3604668, 590013, 1425594, 1425595, 2228416, 1425599, 1425600, 2670786, 4456644, 2801861, 2228422, 2228424, 2883784, 770249, 2228427, 2228428, 2883788, 4456654, 2228431, 4948173, 2228433, 4456658, 2228435, 4456659, 2883798, 2883799, 1507544, 2228440, 2228444, 2883806, 2883807, 2228449, 2228450, 1507555, 2228451, 2228452, 2228454, 2228455, 295144, 2228456, 2228457, 2883810, 2228460, 1097958, 1852965, 2228467, 2228469, 2228470, 2228472, 2228473, 295165, 2228478, 2228480, 2228481, 2392323, 2228484, 1802501, 2228486, 2228488, 2228489, 2228490, 2228491, 3571979, 2228493, 2228496, 2228497, 2228502, 2228503, 2228505, 2228508, 2228511, 3572001, 917794, 2228514, 295204, 2228516, 3572005, 721191, 2228519, 2228520, 2228521, 3572006, 2064684, 3572007, 3572008, 3572009, 3572012, 3572014, 98610, 3572015, 2228532, 3572016, 983350, 3572017, 2064696, 3572018, 2228538, 3572019, 2228540, 3572020, 2228542, 586222, 2228544, 2228545, 2228546, 2228547, 2228549, 2228550, 2064714, 721227, 1311051, 2228554, 2228556, 2228557, 2228558, 2228559, 2228560, 2228562, 2228563, 2228567, 721241, 1376602, 2228573, 2228575, 2228578, 2228579, 2228580, 2228585, 2228588, 983407, 1245551, 2228592, 721267, 2228597, 2228598, 721271, 1245559, 2228601, 2228603, 2228604, 721278, 2228607, 2228608, 721281, 2228609, 2228611, 721287, 721289, 721290, 721291, 2228620, 2228623, 2228624, 2228625, 2228626, 2228627, 999826, 2605461, 721303, 2228632, 2228633, 2228634, 2605468, 2228637, 2228638, 2212255, 2228640, 16802, 885155, 721318, 2228646, 2228648, 2228650, 2228651, 721324, 721325, 2228652, 2228653, 2228654, 2228656, 2228657, 2228659, 1524145, 2228661, 2228662, 2228663, 2228668, 721342, 721343, 721344, 2228670, 721346, 2228674, 2245057, 721349, 2228677, 2228678, 2228680, 2228682, 2228685, 2228686, 2228688, 721362, 2228690, 2228695, 2228696, 2228697, 2228701, 2228702, 2228703, 3572190, 2228705, 2228707, 1311204, 2228710, 2228711, 2228712, 2228713, 2228714, 3572202, 2228716, 4522473, 2228718, 2228719, 4522476, 4522477, 2228722, 2228723, 2228724, 2228725, 2228726, 1311223, 1311224, 2228727, 1311226, 2228730, 2228731, 1311229, 2228732, 2228733, 2228734, 2228735, 2228736, 2228737, 2228738, 2228739, 1311238, 2228740, 2228741, 2228743, 2228744, 2228745, 2228748, 2228749, 2228750, 2228746, 2228752, 2228747, 2228751, 2228755, 344595, 2228757, 2228758, 2228759, 2228760, 1311267, 1311268, 1311270, 1311271, 1311272, 344616, 2622003, 2622004, 557622, 1311286, 1311287, 1311289, 2622006, 2622007, 344637, 1311296, 1311305, 4833865, 1311307, 4833866, 4833868, 2474581, 1311320, 1278559, 4833888, 1278562, 4833890, 4833892, 590437, 4833893, 4833897, 4833899, 492142, 590451, 590454, 590456, 590457, 492154, 3195512, 1819257, 3195520, 590469, 1950342, 3506824, 3195548, 377511, 4063915, 4113068, 3195574, 377536, 3195584, 2425547, 377554, 377555, 3195607, 3506904, 3506909, 4276963, 377574, 1114857, 1475305, 1114859, 1295081, 4276969, 377579, 1229553, 4276979, 3506933, 377589, 4276981, 377593, 4276986, 1901309, 4932349, 1950464, 2687745, 4276995, 4276996, 4276997, 2687750, 4276998, 377607, 4932360, 3506954, 3506955, 3506957, 3769101, 3506962, 2687763, 3883796, 2687765, 377624, 131865, 4260637, 4932383, 3195680, 3195683, 4260645, 4113191, 2687784, 2310959, 4473654, 3638075, 4031304, 131916, 4031309, 4932428, 4932430, 1688404, 2048856, 2556761, 279386, 2016090, 279388, 4932445, 2311007, 131940, 951141, 131942, 1229668, 1753956, 4866917, 4031338, 3294055, 4932452, 4932464, 4031345, 2327414, 4932471, 4932472, 4113277, 951172, 4932490, 4031374, 4932500, 1819541, 2687894, 4031384, 525210, 4031388, 3834782, 2687906, 2687909, 4932519, 4932520, 394154, 3474348, 1360816, 4555700, 525247, 4867015, 4867019, 4867020, 2671566, 1360851, 4309972, 4867029, 4309974, 1360855, 1360857, 4867040, 2606050, 4867042, 4867043, 4867045, 525287, 525290, 525292, 4867053, 4867054, 4867055, 3818491, 525308, 4391932, 4867076, 4867087, 4867093, 4867094, 4391959, 4867095, 4736024, 3818524, 4867110, 4867111, 4408361, 4867115, 2802732, 1836090, 4391994, 3818555, 4867133, 4867134, 4162623, 3425347, 3900484, 2196549, 3408972, 4392012, 984142, 3408974, 4392015, 3818579, 4392023, 4244584, 4588658, 4441204, 3900537, 3900538, 3425412, 2196615, 3179662, 2933904, 3425425, 3425429, 1885336, 4818075, 3900582, 115886, 2458810, 3900611, 4818124, 3474641, 3474642, 4424920, 3900633, 132318, 3900641, 3474658, 3900643, 3900644, 3900645, 34023, 3900648, 3900650, 3900651, 3900652, 3900653, 3900657, 3900659, 17652, 17653, 787702, 3474678, 3900662, 3900663, 3589369, 4408571, 17660, 17665, 17666, 17667, 3900677, 17670, 17671, 17672, 886025, 17673, 754955, 754957, 754959, 3900688, 3589395, 3900692, 3900693, 3900694, 3900695, 3900696, 3589397, 754974, 3900704, 3589408, 3900706, 3589409, 3589410, 4719911, 3900712, 3900713, 3900714, 3900717, 3900722, 3622196, 3900725, 755003, 3900732, 3900734, 755009, 3900737, 1328449, 3786062, 1885523, 3900762, 3900763, 3900769, 755049, 755050, 3900779, 755055, 4228472, 3900794, 3900797, 3900800, 3900801, 165257, 2557321, 2786699, 755084, 3900810, 755087, 1328543, 1328549, 1328555, 165292, 4801966, 1557939, 755127, 1557950, 755138, 755140, 755141, 755142, 1852881, 4195802, 2442716, 1852893, 1181156, 1852903, 1852904, 3278317, 1852909, 1279471, 4162042, 165370, 165376, 1852933, 165384, 4670987, 165388, 3491342, 1852944, 1852945, 1852946, 1852947, 165397, 165398, 1852949, 1852950, 1852951, 1738266, 2082327, 165404, 1852953, 1852954, 3835423, 1852955, 3868193, 2426402, 3868195, 1459748, 1590821, 165414, 1852964, 968231, 1590824, 1590825, 3671595, 4752939, 4752941, 3835438, 575018, 1885740, 2410028, 2410031, 2410032, 2410033, 1885756, 1885763, 3671621, 3835462, 4785735, 1885776, 1771091, 1427028, 1885781, 1771094, 3229269, 1885787, 3835492, 3835495, 2098797, 2098799, 2098800, 2098802, 3835509, 99958, 4556406, 2098808, 3229305, 2098810, 3229306, 3835516, 3229307, 1803905, 2098820, 3737220, 3229317, 3229318, 2098824, 3229319, 3835530, 3229320, 2754192, 3835538, 3835551, 3835553, 1017506, 1427106, 1427107, 1427109, 3835558, 3835560, 3835561, 3835562, 3835564, 3835566, 1410749, 1410759, 3540686, 3540690, 3540698, 4671213, 788207, 4458246, 4409094, 673563, 2803492, 2803495, 2950995, 4409177, 2951002, 378721, 2951021, 2951025, 4409205, 4720504, 4720505, 3835776, 4409217, 1492866, 1984411, 1984412, 3639197, 886722, 2377685, 2852828, 2951143, 2492407, 34818, 3901444, 4343819, 3164183, 3164191, 2459687, 3311668, 1099835, 1460289, 4524110, 4524113, 4524114, 4524115, 1034322, 165973, 4816709, 3770480, 1575033, 4638848, 1837197, 3393681, 4638867, 772249, 772250, 133276, 3475613, 559305, 1476823, 1509601, 4753640, 4753645, 4671736, 575745, 3066115, 3066116, 2394397, 3049761, 2224618, 1050915, 4753701, 2394412, 2394413, 2394414, 2394415, 1722670, 3279159, 3279160, 411959, 2410813, 3918145, 4671815, 411983, 411984, 411986, 411992, 412002, 2230627, 412003, 412005, 412006, 412007, 412008, 412009, 4508005, 4508006, 2230637, 2418, 1640819, 2230647, 788861, 4671877, 1051014, 2230667, 1051020, 2230668, 1051023, 1051025, 1051034, 1051041, 1051046, 1051053, 2935215, 1051057, 362944, 3885513, 2034127, 2034132, 2034133, 2034136, 2034137, 739804, 4622819, 739812, 625131, 625141, 2836982, 625143, 625145, 625150, 2787840, 625158, 2902540, 2902541, 2902542, 625168, 2902544, 2460185, 625179, 1804834, 2837026, 3017253, 2312741, 641578, 2034252, 461389, 2116181, 2116182, 2853464, 4557416, 3439338, 3963628, 2820774, 3705513, 871084, 3787436, 1886894, 3705520, 3787444, 3705528, 3787453, 1690302, 1379008, 3705536, 3312325, 3705545, 1772239, 1690319, 1690338, 3312359, 84711, 1690345, 2788076, 1215212, 1690349, 1690350, 4983536, 772850, 3967735, 3967740, 1690364, 1690365, 543489, 1690371, 1690372, 1690373, 2591497, 1690377, 1690380, 1690381, 1690382, 1772303, 1690383, 1690386, 1690387, 1690389, 1690391, 1690394, 1690398, 1690399, 1772321, 248614, 1690406, 248616, 1690409, 1690411, 1690412, 1690413, 1690414, 1215280, 1215281, 1215282, 1772338, 1690416, 1690417, 1690418, 1690425, 3394367, 4508485, 101194, 2394957, 887632, 101203, 101209, 101214, 2935650, 4983658, 1215350, 1215351, 1215352, 4983670, 4983676, 1215370, 4983696, 248720, 1215379, 4983700, 1952659, 1215386, 1280926, 4967331, 3361702, 609191, 4983722, 3885994, 2624430, 4983727, 2624437, 2624442, 2624445, 4983744, 1543108, 1215433, 4983753, 1215438, 2100181, 1215446, 4541402, 4541405, 4656095, 4541409, 2149347, 2149349, 2149350, 4492263, 4541413, 4541414, 2149353, 4639726, 1084399, 150511, 2149361, 3165170, 150516, 1084407, 822266, 822267, 2149371, 822269, 822270, 150525, 1084417, 822277, 4426757, 4426758, 2149382, 2395156, 2149400, 2149402, 2395174, 2395179, 85044, 2228032, 1297477, 1477707, 822359, 3230812, 2228040, 511078, 2362476, 150643, 150644, 1100928, 1707152, 4721824, 838821, 4721842, 3443891, 3443892, 4721843, 2788534, 4721844, 4721846, 4721847, 4148406, 4213943, 4148409, 2788542, 4934850, 2706629, 822470, 2788550, 3624136, 2788557, 2788570, 4951264, 2788577, 2788578, 1854688, 1854689, 1854691, 1854692, 2788583, 4705510, 1854697, 1854699, 117998, 2788597, 2788600, 2788606, 1772800, 3624195, 2788616, 3132681, 2788619, 2788622, 2788623, 2788624, 2788626, 970008, 970014, 3689767, 4443431, 3378474, 118059, 3362092, 118063, 118075, 118085, 3902790, 4394314, 2706765, 2706766, 2706767, 68950, 4443486, 118111, 3542378, 118122, 118123, 118127, 118128, 3378550, 3378554, 118139, 4443517, 3329408, 3329409, 4443525, 4345224, 4443529, 3132812, 4443536, 4443538, 4345235, 4345248, 429473, 3378594, 3050916, 1002920, 462249, 429483, 462255, 1232303, 429489, 1904049, 4443570, 4443571, 1232309, 2969014, 429495, 691640, 4443575, 429500, 691646, 429505, 429506, 429507, 3378628, 1232322, 429510, 1232324, 4443589, 4443590, 4443593, 429515, 1232332, 4443596, 4443597, 4443599, 3640789, 1232341, 429527, 429529, 429530, 1232348, 3378655, 429536, 560608, 560610, 3968483, 3378660, 3968484, 3968485, 1232354, 3378664, 429545, 3378665, 1232356, 3378668, 1232357, 1232360, 3378671, 1232361, 3378674, 429555, 3378675, 3378676, 3378677, 3378682, 3378683, 3378684, 3378685, 3378687, 3378688, 3378689, 101887, 3378691, 3378692, 3378693, 3378694, 429567, 3378696, 429570, 3378698, 3378699, 1379852, 3378702, 3378703, 3378704, 1379855, 3378706, 3378707, 3378708, 3378710, 3378712, 3378713, 3378714, 3378715, 101917, 3378717, 3378718, 3378719, 3378721, 3378724, 3378725, 3378726, 3378727, 3378732, 4329008, 4984370, 3378739, 101940, 3378745, 3378746, 4886075, 4984379, 3378749, 642620, 4886079, 4886080, 3378754, 3378755, 642627, 4886085, 101958, 3378761, 3378764, 3378765, 3378766, 4984396, 4984398, 642639, 1314388, 101975, 4886106, 101980, 4755036, 101982, 4755039, 642652, 3395167, 101986, 642656, 3395169, 642659, 3395174, 3395175, 3395176, 1724016, 4820599, 4492920, 1887866, 4640385, 4247176, 4132491, 4132496, 691859, 691863, 3952282, 4640413, 691870, 3018398, 1887904, 4640416, 233128, 691887, 1887920, 1887926, 4296375, 691897, 691905, 4296385, 2723523, 1887941, 233160, 1887945, 2887372, 2887373, 4640462, 1887952, 85713, 85714, 1281747, 85716, 1887963, 1887965, 2477789, 2477790, 233185, 1887970, 233190, 560870, 560871, 560872, 233194, 1887978, 1887979, 1887980, 1887982, 233199, 233203, 1117947, 233213, 233220, 233222, 1412874, 3952411, 4689692, 1429277, 4329252, 4689709, 1806130, 675642, 675643, 675644, 1494844, 610113, 1806151, 2395976, 3182407, 4034386, 3854162, 1658720, 1658725, 3493749, 3886966, 397175, 3493752, 3886968, 2658200, 4149159, 2428852, 2428853, 610229, 4542390, 741309, 2428863, 741321, 1069005, 4542413, 1069008, 1069017, 1363929, 4542431, 1347552, 1363939, 1069033, 1363945, 4542442, 1363948, 1363950, 4542446, 4542448, 1052658, 4542450, 1363955, 1363956, 1069045, 4542453, 1363971, 315396, 315406, 1069075, 921625, 4427803, 938019, 1314855, 3067966, 4001856, 2854981, 561233, 4198488, 3067994, 3067997, 561251, 938091, 1708148, 1708149, 1708151, 3166346, 561291, 4313248, 3149992, 561329, 757943, 757947, 757948, 757950, 2330814, 757953, 2330823, 757960, 561353, 3772617, 3363027, 3363036, 3363048, 3363050, 3805419, 545016, 3690748, 594177, 594178, 4165891, 4165894, 3608841, 3363100, 3363102, 3363103, 2986277, 4755770, 3002682, 332092, 3363130, 807230, 2363714, 3019075, 332100, 3363140, 2576709, 332103, 2576717, 3363149, 528726, 528730, 4755804, 4755805, 1479009, 2576740, 3641703, 4755819, 1479021, 1479022, 1479023, 3641711, 3740015, 4723055, 4755822, 4755828, 4755829, 4755830, 4755831, 4755832, 2576762, 4755835, 4755836, 4755838, 4755840, 4755841, 4755846, 4755849, 4755850, 4460939, 4755852, 4755853, 2576777, 2576779, 2576781, 2576783, 2576784, 4723091, 2576789, 4723096, 2576799, 2576800, 2412963, 2576803, 2576806, 2576809, 3690922, 2576812, 2576817, 2412978, 2576820, 2576821, 2412982, 2576751, 2576823, 2576824, 1774010, 2576825, 2576826, 2576827, 1249726, 2691528, 1872331, 1872332, 1872336, 4510161, 2953691, 2953703, 1249771, 168438, 168445, 201224, 201228, 201229, 3199538, 3199539, 3199540, 3084865, 4870725, 4870729, 4870734, 4149840, 1036890, 2380380, 1282653, 1282656, 4035170, 1282659, 2462309, 1430121, 4149868, 4149870, 4149875, 4149881, 3707514, 4149882, 4149883, 4149884, 1282686, 4756094, 3412609, 3412610, 4817227, 4035204, 2118278, 1282695, 316039, 1282697, 1282698, 1282699, 1282700, 1282701, 1282702, 4166311, 201387, 1217198, 201397, 2036411, 1544896, 1544897, 201410, 742083, 3035851, 2036431, 2020051, 2036440, 2036445, 2790117, 398059, 2839281, 2790134, 3920635, 3920636, 2790146, 594691, 2790147, 2790148, 1823509, 1200928, 1069862, 168764, 987977, 2724690, 4969303, 4969304, 4969306, 1834808, 4969310, 4969312, 4838245, 3445616, 3445619, 349049, 3543934, 3543935, 4789148, 906152, 1774523, 2872263, 2872265, 2872268, 2347982, 545742, 2872281, 1627099, 1627100, 2872285, 1577956, 2413540, 1872872, 4117487, 1250292, 693248, 1807361, 1807364, 775172, 775175, 775176, 775177, 4379667, 2331669, 2495527, 4576295, 4609064, 250927, 2217011, 3298359, 250937, 3298375, 1889352, 1889354, 1889355, 1889356, 1889357, 1889358, 1889359, 1889360, 1889362, 1889364, 4166766, 4166767, 4166776, 676986, 3118204, 2069636, 2315421, 4969631, 2282666, 2774215, 3396807, 1643738, 1905891, 2774244, 2774246, 464110, 1873135, 2299123, 4592886, 4592890, 2757885, 3183878, 3183885, 2561298, 2561299, 3183890, 3183892, 513306, 529702, 2577702, 3183921, 3183927, 3593528, 4592953, 2561339, 2561340, 3183937, 3183963, 3183965, 3183966, 2774367, 2774368, 4789618, 1693058, 3282314, 1775035, 1103292, 1168830, 3282375, 3315148, 3315149, 3315150, 3282383, 3282405, 4101617, 1578484, 4101625, 3282428, 4101629, 4101631, 4101634, 4101635, 4101638, 2954770, 4101675, 2627140, 3642951, 2627148, 4740688, 1889884, 1889897, 1955437, 1726086, 202376, 3380886, 1791654, 2381480, 2807469, 2807473, 186036, 4265671, 3544784, 2627282, 3544788, 4265691, 1644274, 3102456, 3102468, 366350, 366352, 366370, 2856746, 1677100, 2840364, 2856754, 4396876, 3626838, 3626839, 661336, 3839837, 2070372, 3725162, 3757938, 2070391, 3364727, 2070393, 3757945, 2225348, 4806529, 2971525, 2955145, 2971534, 2971538, 4954003, 2971539, 4806548, 2971542, 4806555, 2971548, 2971549, 2971550, 989089, 3757987, 3757988, 3757989, 4806564, 2840507, 317376, 4806592, 4806594, 2840520, 2840531, 2840537, 4102106, 4806618, 4102108, 2840541, 4806622, 4806624, 989157, 3332070, 989162, 317436, 317437, 317441, 2168835, 4839428, 989195, 317452, 2840592, 317460, 1775638, 1267735, 2152474, 2168859, 2316314, 317468, 317470, 776225, 989221, 1775654, 989223, 2168870, 2840614, 2840615, 2840617, 989228, 2840623, 1054772, 694325, 4528181, 2840634, 2840635, 989244, 989246, 989247, 2840639, 1775681, 3921984, 1775684, 989253, 989254, 989262, 1185884, 1185887, 989290, 989291, 989293, 2332782, 989295, 4888688, 989297, 3561583, 3561584, 989302, 989304, 989306, 989311, 989312, 989315, 989316, 4774021, 989319, 989320, 989321, 989323, 989324, 2414732, 4774031, 1104021, 530582, 989335, 4397207, 989337, 2414741, 989339, 2414743, 4774037, 3987609, 4774040, 989344, 989345, 989346, 2414749, 989353, 989357, 989358, 4626606, 4626607, 4626608, 989362, 989363, 4626609, 4626610, 4626611, 989368, 989369, 989371, 989374, 989380, 989381, 989382, 4905158, 989384, 71883, 989388, 989390, 989391, 2906325, 137430, 4905175, 4610267, 4905184, 4856034, 4905188, 4905189, 4135143, 4086000, 4905201, 4905207, 3889400, 2906367, 1939715, 1513736, 2316559, 2316567, 2316569, 2087201, 2070819, 2087204, 2087205, 2087206, 2087207, 153910, 1399095, 1218872, 1399096, 1399097, 1399098, 153911, 153913, 1218878, 153914, 1218880, 2103616, 153915, 1186115, 153917, 153918, 153919, 2906434, 2103625, 2103626, 1218892, 1186131, 4790613, 1186137, 1186140, 1186141, 1333601, 4741474, 2906470, 1333610, 1333611, 1513836, 2906482, 2759032, 2906488, 2906494, 1923457, 1923458, 4643209, 1055118, 350618, 3840429, 3840433, 1513909, 2906549, 1513918, 4413887, 2906563, 1513925, 121287, 4807114, 3938763, 1513932, 1513934, 563663, 563664, 1513936, 563666, 661970, 563668, 1513938, 1513939, 1513941, 1513944, 3938768, 4807122, 1513947, 1513948, 1808860, 1513950, 2906587, 121310, 2742751, 2906592, 2906593, 4250081, 4250082, 1513958, 1464805, 2906597, 2906598, 1464808, 1513969, 1513970, 2071026, 3791349, 1874427, 3906045, 1137149, 1235455, 1235482, 1235483, 3791393, 121379, 121381, 121382, 121383, 3545643, 3545644, 121388, 367151, 121391, 3545649, 3791407, 121396, 1628725, 3185263, 23171, 776843, 400025, 400035, 2218659, 23213, 252589, 3185328, 875188, 4938426, 170683, 23227, 252605, 170687, 252608, 3185346, 3185347, 4938435, 4938436, 3185353, 3185354, 4004560, 3283667, 4004563, 3283669, 23262, 4004576, 3463914, 4791022, 252657, 1694452, 711417, 2595583, 2431747, 3087117, 2808616, 1694514, 3087156, 2349876, 2808630, 2808633, 1186618, 3087164, 2349886, 760639, 2349887, 2071363, 2071364, 2071365, 613189, 760647, 3496773, 629586, 3365715, 3365716, 3365720, 3365722, 3529564, 1694560, 908129, 1694561, 1694562, 1694564, 3365732, 1694566, 3365734, 2726760, 1694568, 2579307, 3365741, 2431857, 2431858, 3365746, 3365754, 1694588, 3677053, 2431870, 2431871, 2431873, 2431876, 2431877, 3365765, 2431879, 2431880, 2431882, 3365770, 3414925, 3365775, 2431897, 2431898, 2431899, 2431900, 2431901, 2431905, 2431907, 2431910, 2431915, 2431916, 2431919, 2431922, 2431923, 2431926, 2431928, 2431929, 2431930, 2431931, 2431932, 2431933, 2431934, 2431935, 2431936, 187326, 629700, 2431940, 2431944, 2431947, 3742668, 351179, 1121232, 2431954, 2431957, 2431961, 2431965, 2431967, 3693535, 2431970, 2431972, 220135, 629737, 2431977, 2431981, 2431983, 2431984, 2431992, 2431993, 2792444, 1022973, 2432001, 2432002, 2432003, 2432004, 2432005, 2432006, 1416193, 3775491, 3775495, 4152321, 2219010, 3775500, 351243, 4234254, 2219019, 2219022, 2219023, 2219025, 3775507, 3775508, 3775509, 4234262, 4545553, 3775512, 3775513, 3775514, 3906584, 2219027, 2219028, 351254, 2219034, 2219038, 2219045, 2153519, 1416240, 1416241, 1416242, 1416243, 1416245, 1416246, 1416247, 1416248, 629819, 1416252, 629821, 1612863, 4299842, 629827, 2825284, 121922, 4054083, 629832, 4299848, 1612876, 203854, 121935, 1072209, 105558, 1399895, 613464, 793698, 2792549, 2382955, 2874480, 2874482, 2071671, 1481851, 2808956, 875644, 1367165, 515199, 2071682, 1367170, 2071685, 2071687, 1399983, 777413, 1055945, 613579, 613580, 892110, 3120335, 892112, 4594894, 1055954, 892117, 892118, 1055962, 4594907, 3120349, 3120350, 1055970, 1055973, 3185895, 1055977, 1400041, 1055979, 1055980, 4726001, 1400050, 679156, 1055991, 613629, 1940737, 3808513, 2579719, 3808530, 613654, 613655, 613656, 613657, 613659, 3120412, 613660, 3808544, 1809698, 1269026, 3808554, 3808556, 3808559, 3808571, 3808574, 1842496, 2170179, 3808579, 3808581, 204130, 3874156, 1252733, 2792842, 1252750, 1252751, 2104725, 3874198, 1170852, 2104744, 4365744, 3874226, 417209, 3644859, 3874236, 3874238, 4332991, 3874240, 4332995, 3644869, 2104779, 3874251, 4333010, 3988963, 4333041, 3988980, 417269, 3988981, 3988982, 4333048, 4398589, 3644929, 3644931, 3988997, 876040, 4398605, 1285658, 4398629, 2842153, 4398636, 1285682, 4398644, 2399808, 56904, 3251789, 2022989, 908879, 941649, 3563090, 941651, 597588, 941654, 56920, 3120729, 941658, 941660, 941661, 941663, 941664, 941667, 941670, 941671, 2432618, 3251818, 3251823, 3251824, 941681, 941683, 3251835, 56955, 3251843, 2842243, 3251846, 2842246, 3792519, 2842248, 1728139, 2432652, 2432654, 2432656, 2432657, 2432659, 2432660, 2432661, 2432662, 2432663, 3792533, 2432665, 2432666, 1236641, 908968, 2842280, 3071659, 2006700, 2842283, 2842288, 908997, 302794, 827097, 1892061, 3514085, 827110, 2072294, 3514086, 2072297, 3776231, 2072303, 2072306, 2072311, 909050, 4660990, 909060, 909067, 2088717, 909072, 909077, 1007381, 909080, 2826012, 2645795, 909093, 1220390, 2858795, 909102, 909105, 909115, 909116, 909120, 909122, 4349767, 909136, 909139, 909140, 909141, 909144, 909149, 3940192, 909152, 909155, 3497828, 909158, 909161, 909166, 3759983, 909168, 909172, 909173, 909175, 4005752, 1007481, 1335163, 1302397, 4349822, 4988805, 4038536, 4988810, 4349838, 4349842, 909203, 4251541, 4349849, 909211, 1007515, 1007518, 1007519, 909217, 1007521, 1007522, 1007524, 4349857, 2564006, 4710309, 909223, 909224, 909225, 1007527, 4349867, 1007533, 909230, 1007534, 4300720, 1007535, 1007536, 1007538, 270261, 909239, 2564024, 909242, 909245, 4349885, 778175, 1007551, 909251, 909252, 4349893, 909254, 909255, 3219400, 2138057, 4349895, 1908683, 4349896, 909258, 1007563, 909260, 909261, 1007565, 909263, 4349898, 909265, 4349903, 909267, 1007571, 4349908, 1007574, 4349910, 1712091, 106460, 1007577, 1007578, 4349914, 1007584, 2809825, 1007585, 1007587, 4349923, 434149, 1007589, 1007590, 2449383, 1007591, 1007592, 57322, 1007593, 4349924, 4349926, 4349927, 4349929, 4153328, 4349932, 4349933, 4349935, 4349939, 4153336, 4153337, 4153340, 401406, 4513798, 1269772, 860175, 1269776, 1269782, 2482199, 2482200, 2498585, 1269786, 1269789, 434210, 2449453, 2416689, 4251700, 434230, 3743801, 3743802, 3514429, 1826883, 2498628, 1826884, 2498630, 2498643, 2498658, 2400360, 2498664, 2498666, 2498667, 2596971, 4153450, 2498671, 2498672, 2449519, 2498674, 614514, 778356, 2449524, 2498680, 2498682, 2498683, 270465, 2498696, 614536, 2498698, 2498699, 2498702, 4792462, 2023575, 2597028, 2597029, 2023591, 4710567, 4710569, 2597034, 2597040, 2597042, 2629811, 2597047, 2629815, 2023611, 2597054, 2023617, 4169922, 1319109, 2597062, 2597063, 2597064, 1319117, 1925329, 3580114, 2597076, 2597077, 2597079, 2597080, 2023642, 2597085, 1333559, 2597087, 2023648, 2597089, 2597090, 2629863, 549096, 3809514, 549099, 3809519, 3809520, 3551207, 2531597, 4514062, 2842909, 1843490, 4120876, 3121456, 2531642, 4120891, 2531645, 4120893, 3678527, 4120897, 4120898, 3350863, 4366679, 1843544, 1843546, 1335645, 1335646, 1335651, 450919, 1335658, 1335669, 1335670, 1335674, 745852, 1335676, 1335678, 1335679, 1253767, 3318174, 3367328, 3318180, 745894, 909735, 369067, 3563947, 4186540, 3563954, 3203507, 3318202, 3563963, 3318207, 745944, 3563992, 3924443, 57827, 745958, 369128, 57833, 1303019, 745978, 4039164, 745980, 745981, 3924478, 745983, 4039171, 745991, 3465737, 745996, 1991184, 746003, 746008, 2761244, 746013, 746014, 3383841, 2761249, 746018, 1991205, 3138086, 3875367, 3138087, 2761257, 1991210, 1991212, 1991213, 746030, 3138094, 746032, 746033, 3138096, 1991220, 746038, 4186678, 2089543, 746056, 4186698, 746060, 3072589, 746062, 4186706, 746067, 3072600, 4186712, 3072607, 746081, 4956770, 4874850, 3760737, 746083, 746084, 4956775, 746085, 4186725, 4186727, 4186728, 4186729, 746090, 4186731, 4776550, 4186735, 4186737, 4186738, 1565299, 4186739, 746105, 746107, 4579967, 746114, 2466436, 2466437, 4203153, 746137, 746140, 746141, 746142, 3187362, 3187368, 3383976, 4580014, 3383988, 3383989, 1155766, 3383991, 3007157, 2417334, 4580023, 4580024, 746170, 4580026, 4580027, 1041086, 3007167, 746176, 746178, 2417350, 3007176, 3269323, 352972, 3384020, 2450133, 4006616, 746203, 4334302, 746210, 3384035, 746211, 3449574, 746214, 4006632, 1336039, 3449578, 746216, 4006636, 3384045, 746218, 3384047, 2908910, 1336047, 2908915, 4334325, 4006646, 4006647, 58105, 1336058, 58107, 1336064, 1336068, 41735, 385802, 1336074, 1336077, 1762062, 1336080, 4514579, 3007252, 3007253, 3007254, 3007255, 1336088, 385817, 3007256, 3007257, 3007258, 3384093, 2450214, 385836, 4416301, 4170542, 3384111, 4170544, 4170550, 385846, 1860407, 2974525, 4416319, 385863, 385871, 3384144, 3384148, 4416340, 1155926, 3269471, 385888, 533346, 533347, 385896, 1565555, 3384181, 2024310, 3384183, 385913, 877433, 3056507, 2024313, 3056510, 385918, 3056513, 385921, 3056517, 3056518, 3056519, 74633, 3384202, 4432779, 3056524, 385930, 3056527, 385935, 385936, 3056530, 385939, 2073492, 385940, 713634, 713636, 3662770, 1844149, 1844155, 4252603, 3007421, 3007422, 3007423, 1844161, 1844169, 1844171, 1844172, 1844177, 1188837, 2745336, 1188864, 1614849, 2663426, 1893378, 1188868, 1188869, 3892227, 2663433, 3892236, 3187737, 3187738, 1336348, 2647073, 4236325, 4138031, 2647100, 3892284, 3892288, 2253903, 2647124, 3892310, 4236381, 4514910, 3892320, 2647155, 3826804, 4138102, 3957884, 1008767, 2106506, 1008778, 1008779, 1008780, 2991248, 1008784, 1008785, 1008789, 3810458, 1008794, 1008795, 4138152, 1008808, 3056810, 3056811, 402604, 3056812, 3056813, 1008810, 1664172, 4777133, 4514990, 4514993, 4662452, 4514994, 2647223, 4777143, 2647227, 4662464, 4662466, 4662467, 3548356, 4662471, 4662472, 3548361, 3548362, 4662473, 4662474, 4662475, 4662476, 1008843, 4777164, 877773, 3548370, 3548371, 3548372, 3548373, 1893582, 1893583, 2647248, 3548377, 3548378, 3548379, 2647252, 2647255, 2647258, 2647265, 1008867, 4236516, 1008870, 3957992, 2647278, 1090811, 2647294, 4269312, 1402121, 2647305, 1991949, 2647309, 2647315, 1991966, 4384040, 4842792, 4384060, 1058116, 4384070, 1058121, 1058122, 3892556, 2663759, 2663763, 2663769, 4384091, 1058143, 4384104, 4695401, 4466026, 1058160, 1058167, 1058179, 1058182, 1058188, 2483596, 2483597, 2483598, 1058192, 2483599, 3925392, 1058200, 2221464, 2614683, 583072, 583073, 2221476, 2221477, 2221478, 2221480, 2221482, 4023722, 2614702, 2909619, 2614709, 2614711, 3466687, 1828295, 173517, 3466701, 1795557, 2909669, 2909670, 3466725, 1893865, 2909677, 1631726, 2909679, 2041329, 3171826, 1893876, 2123255, 2909690, 3696123, 2041348, 3696133, 4990474, 3696141, 108046, 3204623, 1549846, 3696151, 2123289, 3696153, 1549851, 4679195, 3696160, 3696161, 3696164, 3401253, 4826663, 3696167, 3696168, 3696171, 3696173, 3696175, 3696177, 3696178, 2041396, 3696183, 3483192, 2041401, 2795072, 435779, 3696197, 3696199, 3696202, 3483213, 3696207, 3696208, 3696212, 3696214, 3696215, 3696216, 2041433, 3696220, 4286044, 3696222, 3696223, 3696224, 3696226, 3696227, 3696228, 4515426, 3696231, 3696233, 2336363, 3696237, 3106414, 3696238, 3696239, 3696243, 3696244, 3696245, 4990583, 3696247, 3106425, 3696249, 3106433, 435842, 3696257, 3696258, 3696261, 4105862, 894599, 435849, 4990606, 3696273, 3696274, 3696276, 435861, 386709, 386710, 4859545, 3696281, 386715, 3696284, 3696285, 3696288, 3696289, 435877, 435883, 1566390, 4613814, 4613815, 435902, 435906, 435915, 2746064, 3581655, 435929, 354014, 354017, 435951, 42738, 435963, 452351, 435968, 435970, 452354, 435972, 452356, 435974, 452363, 4777740, 452365, 91919, 435984, 452369, 452371, 435988, 435989, 4777749, 452374, 2041624, 714525, 4777758, 976671, 4777759, 91937, 436002, 4597538, 4777760, 4597542, 1468200, 2565929, 1468201, 436015, 436022, 436023, 436024, 894789, 993107, 1681237, 91996, 42845, 4925282, 42851, 42860, 4401005, 42863, 42868, 583540, 4401016, 4237184, 4401027, 993162, 4401035, 3205007, 993169, 3843987, 993172, 4155293, 993184, 1992617, 3024810, 4171692, 993197, 1927084, 1927085, 3008428, 3008429, 993205, 4270005, 993208, 4401080, 4401081, 4401084, 4171712, 780230, 4171725, 2172878, 4073441, 3123214, 2598974, 2598976, 2598979, 2598980, 2598981, 1042499, 1861723, 1501276, 1501278, 1501279, 1730669, 4139122, 1861747, 3909751, 1730683, 649341, 2975878, 2975883, 2631820, 2975894, 1976474, 1976488, 1370280, 1976490, 3958954, 2975918, 2681008, 2975920, 1976499, 2336949, 2336951, 2336952, 1976505, 2975928, 3958969, 1976508, 3958970, 1976514, 1976516, 1976521, 1976522, 1976524, 26831, 26832, 1976529, 747730, 1960151, 1976538, 1976539, 4499674, 4499675, 4499676, 4499678, 4499680, 4483291, 1976546, 1042653, 1960164, 1960165, 4434151, 1976552, 1960169, 1550570, 1960172, 1976558, 2631919, 4499697, 1960177, 2451697, 4974836, 1976566, 1976567, 1976568, 1976569, 1976570, 1976571, 1976572, 1976573, 1976574, 1976575, 1976576, 1976577, 1976578, 1976579, 1976580, 4499710, 1976582, 1960196, 1960206, 1435927, 1435934, 1960231, 1960244, 1960245, 3664198, 4614471, 944457, 944458, 944459, 4696398, 1304921, 2009454, 3025267, 2828665, 3942800, 174484, 813501, 3434945, 1042883, 207308, 2468300, 4925904, 420307, 3434965, 4925912, 420314, 207330, 780772, 207334, 207335, 780781, 1550868, 3697173, 1042982, 2320936, 1042991, 4909616, 4991539, 1042997, 4909621, 4909622, 4909632, 1043013, 2320975, 1043024, 1043026, 1043027, 1043028, 1043032, 1043033, 1043036, 1043037, 1043038, 1043039, 1043040, 3779169, 2894434, 1043042, 4369008, 1043057, 1043058, 4483697, 4483698, 2665087, 191108, 2665100, 4516492, 1043088, 4811410, 1043093, 1043094, 2321045, 1043097, 2321050, 240283, 1043100, 3566237, 2271903, 1043104, 2615975, 4156071, 3566251, 2321071, 3418810, 2321084, 2321085, 3566271, 1916966, 3566274, 2796229, 2796232, 1043144, 1043151, 2321103, 2648786, 1043157, 2321115, 1108700, 2321120, 2321121, 92899, 2058982, 2058983, 2321132, 2648812, 2321134, 2321135, 2321137, 2648817, 2321140, 1616630, 2321143, 1043192, 2321144, 2321145, 2648823, 2648827, 4352759, 2894591, 2321151, 2648831, 2321154, 1616643, 2321155, 2648836, 4418307, 2321158, 2648838, 4418311, 1075977, 1616651, 2321164, 2321165, 4172558, 2321168, 1616659, 4418324, 2321171, 1043220, 1043221, 2321173, 1043223, 1043224, 1043225, 1043226, 1043227, 2321179, 2321180, 2321181, 2321184, 4008738, 3369757, 4172583, 4172584, 4533032, 2321195, 2321196, 2321199, 2763568, 3746608, 3746610, 2321201, 2321202, 4533046, 2763576, 3746618, 2321210, 3746620, 2321211, 2321216, 3730246, 2321226, 2321227, 2763599, 2763600, 2321232, 2321234, 2321236, 2321237, 2763606, 2321238, 2321239, 3730261, 2321241, 2321242, 3746652, 2321244, 2321249, 2321251, 2321253, 2321255, 2321256, 2321257, 2321259, 2321260, 2321262, 2321263, 2321265, 2321266, 2321268, 2321271, 2321274, 2321275, 2321281, 3206018, 2009987, 2009989, 2321286, 732039, 4172679, 4172682, 2321295, 2321298, 2321301, 3435415, 3517336, 2321309, 2321313, 3435425, 4172707, 4172708, 879526, 2321322, 4172715, 2616236, 4172717, 338864, 2321328, 2321329, 2321330, 2321335, 2321338, 2321339, 4172736, 3402690, 2321347, 2321350, 4172743, 2763722, 2321354, 4172749, 2321358, 4172756, 2927574, 2321366, 2321368, 2927578, 2927579, 1223644, 2927580, 2927581, 2927582, 2321371, 2321380, 2321381, 3927015, 2321385, 4189161, 2321390, 2321393, 961522, 2321399, 2321410, 2321411, 2321413, 2321420, 2321421, 2321424, 2321426, 2321427, 2321429, 846870, 2321430, 2321432, 4041753, 2321435, 2321436, 2321440, 2321441, 2321442, 2321443, 2321446, 2321447, 2321448, 1633326, 2321455, 2321456, 1272882, 1272883, 1272884, 2321458, 1125430, 1633334, 2321464, 2321465, 2321459, 1633336, 1633337, 1633338, 1633339, 1633340, 1633341, 1862713, 1633343, 2321472, 1633344, 1633346, 1633347, 1633348, 2321474, 4762697, 1535046, 1633350, 1633351, 1633353, 2321477, 765007, 2321486, 3976274, 3976275, 2321493, 2321495, 2321497, 2321498, 2321508, 2321509, 1436774, 2321510, 2321511, 2321512, 2321513, 4058212, 2321517, 2321518, 4631665, 3452020, 2321524, 2321525, 2321526, 2321527, 2321528, 2321529, 2321530, 2321531, 2321533, 2321534, 2321535, 2321539, 2321540, 2321542, 4041864, 2321544, 2321545, 2321549, 2321550, 1862799, 4041872, 2321551, 2321554, 2321557, 1862808, 2321560, 2321561, 1862811, 4041884, 2321564, 3533977, 2321566, 2321567, 4893859, 2321571, 2501797, 2321572, 453797, 2321573, 2321574, 2321575, 3533989, 4451494, 3517613, 2321579, 2321580, 3533998, 2600118, 2600119, 2321591, 2321592, 2321595, 2321597, 2600126, 2321598, 2600129, 2321602, 2321603, 781508, 1387718, 2321608, 453833, 2321617, 2321621, 2321623, 2321624, 2321625, 3927255, 896219, 896220, 2321628, 2321629, 2321631, 2321632, 339170, 2321636, 2321637, 3927268, 3927269, 2321640, 2321641, 2321643, 2321644, 2321645, 3353838, 3927277, 2321647, 2321648, 2321649, 2321650, 2321652, 2321653, 2321654, 2321655, 3927288, 2321657, 2321658, 961787, 4893947, 2321659, 2321660, 2321661, 3353856, 2321662, 3353858, 2321663, 2321669, 2321670, 2226444, 535816, 3353865, 4025612, 2321678, 2321680, 2321681, 3353874, 535827, 4320530, 1420565, 3353877, 535831, 2321683, 830745, 945430, 3353883, 2321688, 2321691, 2895134, 535838, 2321692, 2321693, 2321694, 3353891, 3353892, 2321695, 535846, 2321697, 2321698, 830761, 535850, 535851, 2895145, 535853, 535854, 830765, 3353904, 830770, 3353912, 3321162, 535891, 4173139, 4173154, 1371491, 4173156, 4173158, 4173159, 4992360, 535931, 535935, 535938, 2387331, 2387336, 2583947, 4009357, 2583949, 1813909, 4009373, 2796958, 2583966, 945574, 945575, 3124648, 945577, 945580, 1666476, 945588, 945592, 945593, 945594, 945595, 945596, 3190207, 945603, 4763078, 945611, 945612, 945614, 945615, 1519058, 945619, 224724, 4009430, 945631, 1519073, 945634, 1502690, 945636, 945637, 1519081, 945641, 945646, 945647, 4009457, 4009458, 945650, 945652, 945653, 945654, 945657, 945659, 945662, 945663, 945664, 2387456, 2387457, 945667, 945669, 2387461, 945672, 945675, 945676, 945677, 945679, 945681, 945686, 945687, 945688, 945689, 4009498, 945692, 2240030, 945696, 945697, 1961507, 831012, 831015, 3845672, 1093162, 3845674, 421422, 945712, 945714, 945715, 945716, 3845685, 2223670, 945721, 945722, 945723, 945724, 945725, 3288641, 3845697, 3845699, 945730, 945735, 945738, 3288651, 3288652, 1093197, 3845709, 3845710, 945740, 945742, 3845714, 945746, 3845716, 3845720, 945754, 2502235, 945755, 2256475, 2502238, 945760, 945761, 945762, 945767, 4861544, 945768, 896623, 945778, 945779, 945786, 945787, 945789, 2453118, 945796, 945798, 945802, 945807, 945808, 945809, 945811, 2223772, 3157675, 3747500, 1797805, 3813049, 2223805, 2223806, 1453767, 3157705, 2682577, 2682585, 1748698, 2682586, 2682589, 61156, 1453797, 3993316, 4615910, 1240810, 61163, 1289970, 2682610, 61171, 863989, 61173, 61174, 2682613, 2682616, 2682618, 2502396, 2682620, 1519358, 2682621, 2682622, 2682624, 2682625, 2682626, 2682627, 1797894, 2502407, 2223879, 585487, 2502418, 1453843, 2502426, 3108635, 3108638, 1453855, 4796194, 4157221, 4157223, 1453864, 1453865, 1453867, 2223916, 1453870, 2338606, 1797935, 1797936, 1797937, 3911475, 1797939, 1797940, 3911478, 1797941, 1240887, 1797945, 1453882, 3911483, 3354432, 2338624, 1240896, 585537, 585538, 585539, 585540, 585541, 585542, 585543, 1453898, 585544, 585545, 1240902, 585550, 2502482, 1453910, 2584408, 2879320, 2584410, 2502501, 1453926, 470893, 2502510, 1175412, 2666357, 2502520, 814970, 814975, 814976, 1453964, 2224012, 1208205, 1208206, 1961879, 2584477, 2043806, 3092383, 2502560, 2600866, 2502567, 2224042, 2273196, 4206512, 2502577, 1454003, 3764147, 2502584, 3551160, 3764154, 2502587, 2502588, 2502589, 3551166, 2502591, 2502592, 1454017, 1454018, 3551171, 1454020, 1454025, 2224075, 1454028, 2502604, 2502605, 4698062, 4698064, 2502610, 2502611, 2502612, 2502613, 2502620, 1454045, 1060830, 2502621, 2502623, 2502624, 3551199, 1454051, 2502628, 2502629, 1454052, 962534, 1454053, 1454055, 4435946, 2502634, 1454056, 1454057, 1454058, 1060846, 1454060, 3551208, 3551209, 3551213, 3551218, 3551219, 3272687, 3551223, 3551226, 667646, 3256320, 3551236, 3551241, 2830348, 2830350, 3862568, 3551279, 1421361, 3551281, 1912883, 3434748, 1421369, 3813435, 1421372, 3551292, 3551296, 3551298, 4337733, 471111, 471112, 471113, 3551303, 28748, 28751, 3551315, 3551318, 2224216, 2371675, 3436638, 3551327, 2388064, 2388067, 2289764, 2388068, 143462, 28774, 2371689, 2371695, 2224239, 143473, 2371697, 143476, 2502774, 28792, 143482, 143484, 2289791, 143488, 143489, 143494, 1798284, 356503, 2224280, 3895448, 2224285, 2502821, 2617515, 2617516, 389294, 3240114, 3240117, 4845768, 4075728, 4845778, 4698323, 4993240, 4993241, 4042972, 4468972, 520429, 4387054, 1028335, 2224368, 2224369, 4387061, 4320541, 4043011, 4844831, 454928, 2191634, 307475, 192804, 454950, 454951, 3436840, 454955, 454956, 454958, 454959, 2928944, 454960, 454961, 618802, 3436846, 3436850, 864566, 864567, 3436851, 4469049, 3436853, 618812, 618815, 4469057, 4226118, 4043082, 4469072, 4469073, 684368, 4469076, 3928406, 4469082, 209250, 3010916, 2224492, 4043117, 2208120, 4764024, 2224511, 2224520, 127375, 2224530, 2224534, 2224536, 143769, 3633568, 3633570, 3633572, 3633573, 2224552, 3633580, 1536429, 2224559, 2732465, 4764085, 4522478, 4764087, 2208187, 3633596, 2208194, 2847174, 2224585, 291274, 4485579, 2224590, 2208207, 2208208, 2847182, 3092946, 3092947, 3092948, 3092951, 3092952, 569817, 2224599, 3092955, 3092959, 3092961, 3092962, 2208228, 3092965, 3092966, 3092967, 3633636, 3404263, 3404264, 3092971, 3092972, 2208237, 3092973, 3092974, 3092975, 3092976, 586218, 3092979, 1602027, 3092981, 3092982, 3092983, 3404271, 2224640, 2208259, 2208261, 586245, 586246, 2208265, 2224653, 2224659, 2224664, 2224669, 3469854, 586273, 3273252, 586283, 1765932, 2224694, 3469879, 2208312, 2994746, 2224699, 2208318, 2224717, 1454674, 3781202, 1454679, 3043927, 2208349, 2208350, 2208351, 2224735, 2208353, 2208355, 2208358, 2224742, 2224743, 2257513, 2224752, 2224762, 2224782, 3617426, 2257556, 2224793, 2257561, 2257564, 2257566, 1454758, 1454759, 2224807, 3617449, 2257577, 1454764, 3617454, 3617455, 1815216, 3617457, 1815218, 3617458, 3617459, 2224823, 3617464, 3617465, 1454778, 3617467, 1454780, 3453630, 1454783, 3617471, 2257600, 537282, 3617474, 3617475, 3617484, 3453645, 3617485, 3453647, 1815248, 2321175, 2224850, 2224851, 3617494, 537303, 2224856, 3437273, 3437275, 1454813, 537310, 3617502, 3617504, 2224861, 1454818, 2224864, 3617508, 3437280, 3437281, 3617511, 2257635, 2224871, 3617514, 3617516, 1454830, 1454831, 2224880, 2323184, 1454838, 1454840, 2224888, 3453690, 1192703, 4829953, 3453699, 602886, 1454859, 2224907, 4715277, 979728, 2126609, 1225494, 4240151, 2126614, 2159383, 2224924, 504605, 2159390, 1225503, 1225505, 3453729, 3453731, 1225508, 2159396, 1225510, 3453734, 1078054, 2159401, 2224937, 1225515, 1225517, 1225522, 1225525, 979765, 897849, 1225530, 832318, 2224958, 2224959, 832322, 832323, 4289357, 4289360, 4289361, 979794, 979800, 979803, 2224995, 3535715, 95081, 2405235, 2225021, 4551549, 3748742, 4551565, 1962899, 2339749, 2077607, 258984, 2077609, 2225065, 2225068, 258989, 2339759, 422831, 4977585, 1209268, 258997, 2470838, 2225077, 2503609, 1209274, 1209276, 1209278, 1209281, 1209287, 1209288, 2061261, 2782158, 422864, 2601937, 1209297, 1668049, 1209299, 2061265, 2225105, 1209302, 2225111, 1209304, 422873, 1209307, 1389532, 2225115, 2225117, 3093471, 1209311, 1209312, 1324002, 1160165, 2601958, 1209318, 1209319, 2225135, 3716082, 3453943, 3453944, 3716088, 3716089, 3716090, 3716092, 3716093, 3716094, 291831, 2225145, 2225154, 4109316, 2225157, 291849, 3077133, 3077134, 3077138, 2225171, 2225179, 2225184, 3077153, 2339874, 4994083, 3077154, 3077155, 3077161, 324650, 324652, 324653, 2716719, 3077167, 2225203, 2225211, 62531, 62532, 62534, 2962511, 2225235, 275540, 2225237, 2520157, 2225246, 275558, 2225254, 275560, 1668201, 881782, 455816, 1438863, 2225299, 4076692, 1684634, 2225307, 1684645, 1684647, 2225319, 2225320, 799914, 799916, 652460, 652463, 2225330, 4830390, 4830395, 652477, 2225343, 2225345, 652483, 4830404, 4830405, 4830406, 652484, 13512, 2536649, 4830409, 4830411, 4830412, 4830413, 4830414, 652487, 13520, 4830416, 652496, 4830419, 4830420, 2225360, 4830422, 2225362, 2225364, 13529, 4830425, 652503, 4830428, 13533, 4830429, 2225370, 4830433, 2225378, 4830435, 2225381, 4830440, 1570025, 2225385, 4830450, 4830451, 2225397, 4830454, 4830459, 2225406, 4404480, 4830465, 2225409, 2225410, 2225417, 2225418, 2225419, 4830476, 2225422, 2225425, 2225434, 2225436, 2225443, 3077411, 2897189, 1406249, 2225465, 2225466, 2897213, 3749181, 2487616, 2176321, 2176322, 2929989, 2225477, 2225481, 2225483, 2929997, 456015, 2225488, 2225489, 3847506, 3847508, 3847509, 2225496, 832857, 2225497, 2225498, 2930015, 2225504, 2225508, 832869, 2897254, 832871, 2225509, 832873, 832874, 2225511, 2897264, 2897265, 2225523, 2225524, 947574, 2651510, 2651511, 2700669, 2225533, 2700671, 2225535, 2225548, 4093329, 4093331, 2225557, 2225558, 4093333, 2225560, 4093336, 947613, 2225566, 947615, 2225567, 2225570, 898467, 947619, 2225573, 947624, 947625, 947626, 2225576, 2848171, 2225583, 947632, 947634, 947636, 947639, 947640, 1242553, 947642, 947643, 947645, 2684350, 2225601, 2225613, 1242574, 2225615, 2733521, 2733522, 947665, 947666, 947667, 2733527, 2225627, 2733543, 1701354, 2733547, 2225644, 2733549, 2225647, 2225649, 2405874, 2405875, 2733554, 2225650, 2782708, 2225655, 2225656, 2225657, 2225658, 2733563, 2225659, 2225660, 2225661, 2733567, 2225662, 2225663, 2356732, 2225670, 2225671, 2225672, 2225675, 2225676, 2225677, 2225678, 2225679, 2618896, 800274, 3716630, 2225687, 2225689, 2225690, 2225692, 2225695, 1226272, 2225696, 2225697, 2225698, 2225699, 3716645, 2225700, 1226279, 2225701, 1226281, 3716650, 1193515, 3257899, 2225702, 3716654, 2225703, 2225705, 3716657, 2225706, 2225708, 1226292, 2225709, 1226294, 2225710, 2225711, 2225713, 3257914, 2225715, 2225717, 3421757, 1226302, 2225718, 2225719, 2225720, 2225721, 2225722, 2225727, 2225728, 3257926, 3978822, 2225729, 3978825, 2225731, 2225732, 2225736, 2225737, 2225738, 3143242, 2225740, 2225745, 2225747, 1226324, 2225748, 1226326, 2225750, 3257944, 2225751, 2225753, 2225756, 2225757, 3257952, 3257956, 2225764, 3257958, 3978855, 2225767, 3978857, 2225768, 3946091, 2225769, 2225772, 1275504, 2225778, 2225779, 1226356, 1226357, 2225780, 2225781, 3896948, 1226362, 3978874, 2225787, 3978877, 2225788, 3978879, 1226368, 3978880, 3978881, 3978883, 1226372, 1226373, 3978884, 3978885, 1226376, 3978886, 3978887, 1226379, 2225795, 2225796, 1275525, 2225799, 1275529, 1226391, 1226392, 2225815, 1226395, 3978907, 3978908, 128668, 2733727, 1275549, 1226401, 1275552, 1226403, 3978916, 3978917, 1226406, 2225828, 2225830, 1226409, 2225831, 2225837, 1226416, 2225842, 1226419, 2225844, 1226421, 1226422, 2225845, 2225846, 2225847, 2225848, 46779, 1619643, 46781, 128697, 2225850, 2225851, 2225853, 46786, 2225854, 2225855, 2225858, 2225859, 46791, 2225860, 2225861, 2225864, 2225866, 980687, 2225873, 2225874, 2225877, 2225878, 2225879, 2356951, 2225882, 2356955, 2225884, 2225890, 2225891, 2225892, 2225893, 4617955, 2225896, 2225897, 2225900, 2225901, 1259251, 2225907, 2225912, 2225914, 2225915, 2225917, 2225921, 2225923, 2225924, 1259272, 2225934, 2225935, 2225943, 2225944, 2225946, 2225947, 2225949, 2225951, 4716320, 2225957, 2225958, 2225960, 2225962, 2225965, 2225967, 2225968, 4405044, 2225973, 718650, 4545554, 2225982, 2225983, 4545555, 1455941, 2225991, 2225992, 2225993, 2225994, 2225995, 2225996, 1029970, 4405075, 2226004, 2226007, 4405081, 2226009, 4405084, 2226013, 2226017, 2226018, 2226019, 2226020, 2226021, 2226022, 2226023, 4405097, 2226025, 4159338, 1832814, 2226031, 4405108, 4405109, 1505149, 3323775, 4405123, 4405124, 2226053, 2226062, 2226063, 2226068, 2226072, 2226077, 79774, 4437919, 79776, 2226080, 2226089, 2226091, 456621, 3520436, 2340789, 2226100, 2226101, 2226103, 2226104, 2226105, 2226106, 2226116, 4650956, 2226128, 4650962, 2226136, 2734047, 2226145, 2226148, 2226149, 2226150, 2291685, 2881512, 2537449, 4618214, 2373612, 341998, 2226160, 2881520, 2226162, 2373623, 2373624, 2537464, 2226169, 4143099, 2373629, 2373630, 2373631, 2881533, 2226173, 2373634, 2373635, 2373636, 2373637, 2373638, 2881537, 4143112, 2226177, 2226179, 2226180, 2226182, 2226183, 2226185, 2226186, 2226187, 2881546, 2881549, 2226192, 4274196, 2881553, 2881556, 2226198, 2226200, 669721, 4569119, 2226210, 2226213, 4569131, 2226220, 2226221, 4569136, 2226225, 4831282, 4831283, 2242612, 2226227, 2242614, 2226228, 1882168, 2226229, 2226233, 587838, 2226238, 2226240, 2226242, 4569158, 587847, 1882184, 3209290, 2226252, 3700815, 2226260, 4569175, 4569176, 4569179, 3700827, 2226268, 2226270, 3373150, 2226272, 2226273, 3373156, 3602533, 4569191, 2226279, 3111016, 2226287, 2226288, 2226292, 2799734, 2226294, 4569210, 4569212, 4569213, 2226302, 3864703, 3864704, 3864705, 2799747, 4569219, 4569221, 4569224, 4569227, 3864717, 2799759, 4569232, 3864720, 637074, 4569234, 1734802, 3864725, 3504278, 4569239, 3864731, 4569245, 4569246, 3504286, 2226336, 4569249, 4569250, 2226337, 4569252, 2226338, 4569254, 3504289, 2799784, 227492, 1374373, 3864739, 4569260, 2226344, 4569262, 293034, 4569264, 4569265, 4569266, 2226346, 2226347, 2226351, 2226352, 2226366, 2226367, 2226369, 2226371, 2226372, 3504323, 2226374, 2226375, 882888, 2226376, 2226377, 882891, 2226378, 3504327, 915663, 2226383, 2226385, 2226388, 669909, 2226390, 915671, 2226391, 2226395, 2226396, 2488551, 1915112, 2652392, 2423018, 669934, 2488559, 882928, 1046769, 2226416, 669939, 915700, 2226421, 2226422, 1456376, 2226426, 669947, 4864254, 669951, 2570495, 1915137, 2226430, 2226431, 669956, 2226432, 1538307, 2652420, 2226436, 2488582, 2226437, 915720, 2226438, 571661, 669966, 669967, 1456400, 2226440, 2226442, 2226443, 1390868, 669973, 2226449, 1390871, 4110615, 2226450, 669979, 1456411, 96539, 1456414, 96545, 669986, 669989, 1915173, 2226470, 2226471, 4897065, 2341162, 669996, 4962605, 4946222, 2226479, 2898224, 2341169, 2898225, 2226480, 2226481, 2898229, 2226483, 2226484, 2226486, 2226488, 1915194, 1915195, 2226489, 604477, 2226493, 2226494, 2226496, 2226497, 2226510, 2226513, 3062099, 4946259, 2226516, 2226519, 4307292, 1603935, 2226527, 2226528, 4110691, 2996580, 4110692, 2226533, 1603943, 2226537, 4946282, 2226539, 2226540, 2226542, 2226549, 4946298, 2226556, 2226557, 2226558, 4946300, 4438401, 2226561, 2226562, 2226563, 2226566, 1636743, 2226567, 2226569, 4946315, 2226572, 2226573, 4946318, 2226578, 4651410, 2226580, 2226582, 2226584, 2226586, 1636767, 2226592, 2226594, 1604003, 2226598, 2226599, 1604008, 129449, 2226600, 539053, 539054, 2226605, 2226607, 2226609, 2226610, 2226611, 2226612, 2226613, 4340150, 2226614, 899512, 2226615, 2226618, 2226619, 2226621, 2226622, 2226623, 2226624, 4094399, 2226626, 2226628, 129477, 129478, 2226630, 2226634, 4340173, 4340174, 2226639, 702928, 2226643, 2226645, 2226646, 4831703, 4225495, 2226648, 2226651, 2226655, 2226659, 2226660, 2226663, 2226665, 2226670, 2226672, 2226674, 2226676, 2226683, 2898431, 2226694, 1292807, 1292808, 2226695, 2226700, 2226701, 2226704, 2226706, 3572021, 2226707, 2226708, 2226709, 2226713, 2226714, 2226716, 2226719, 2226724, 2226726, 2898475, 2226731, 2226732, 195118, 3062319, 2226734, 2226738, 2226741, 2226742, 2226744, 2226745, 2226749, 2226750, 2226751, 2226756, 2226757, 2226758, 2226761, 195146, 2226762, 2226765, 195150, 2226768, 195155, 195156, 2226771, 2226777, 2226778, 4225628, 3750494, 195167, 1981026, 2226786, 2226788, 2226789, 2226790, 2226791, 2226793, 2226794, 2226798, 195183, 195184, 2226799, 2226800, 2226801, 195196, 2226812, 2226814, 2226817, 2226818, 3652228, 195205, 2226821, 195207, 2226822, 2226823, 2144906, 2226827, 2226830, 2226832, 3062417, 2226833, 1440404, 2226837, 916118, 916119, 2226838, 2226839, 2226840, 2226841, 3439257, 2226845, 2226850, 2226853, 2226854, 2226859, 2341549, 2226861, 2226863, 2243249, 2226865, 2226866, 2226868, 2226871, 2226872, 2226874, 3701436, 3701437, 2226876, 2226877, 654014, 3439292, 637634, 3439296, 2226881, 1735362, 2144966, 1735363, 2079432, 2226883, 2226885, 3439300, 3439303, 2226891, 3439308, 2226898, 3439315, 1440469, 2226901, 2226903, 2980566, 3439319, 3439321, 2276059, 1440476, 2226909, 2538206, 3439327, 2226915, 2538214, 3357415, 1653480, 3439334, 3439335, 867051, 3439336, 2226921, 2226922, 2226923, 2226924, 867057, 2226925, 2226926, 2226928, 3439340, 4668150, 2226933, 4225781, 2226937, 2226938, 948987, 3963643, 3963644, 867070, 2226942, 3963646, 3963649, 2226946, 4029186, 867082, 2226955, 2226960, 3046170, 2226971, 867100, 2226973, 4225823, 2226977, 867106, 867107, 2226981, 3160870, 2226982, 3685160, 2647245, 768810, 2226983, 3160876, 2226984, 2226985, 572207, 2226993, 2226994, 2226998, 1653559, 3046199, 4225850, 2227008, 2227010, 1751876, 3324740, 2227014, 2227016, 2227022, 2915151, 2227025, 2915155, 2227029, 2227031, 2227032, 2227034, 2227036, 2227039, 3046240, 2227043, 1719143, 4193128, 2227051, 2227052, 3046251, 1653616, 2227056, 2227057, 2227059, 2227060, 1653621, 621429, 2227061, 2227062, 621432, 2227065, 2227066, 2227068, 2227069, 1522558, 2227070, 801667, 1375111, 621449, 801674, 4045706, 2227081, 2227083, 2227085, 2227086, 801680, 1522577, 2227087, 2227092, 1653653, 2227094, 2227096, 1653657, 2227098, 4193180, 4193183, 2227107, 4193187, 2227110, 2227111, 4193190, 3423152, 2227121, 2227122, 326579, 4225971, 2227134, 2227135, 2227140, 2227142, 326604, 2227152, 3588392, 326611, 326612, 2227155, 1653718, 326614, 326615, 326617, 2227161, 2227163, 326622, 3259359, 2227167, 4226016, 326625, 2227170, 3259364, 1276899, 2227172, 1276901, 1276903, 1522666, 1276906, 2227178, 1522673, 4226033, 2227186, 2227187, 326644, 326645, 326646, 2227194, 1522683, 4029434, 2227196, 1276926, 1276927, 1276928, 326657, 1276929, 1276930, 1276931, 3259397, 326661, 2227204, 326663, 2227208, 326664, 4226051, 4340749, 326669, 326670, 2227213, 3259409, 326672, 3259411, 3079185, 326676, 3259414, 1932309, 1932310, 1932311, 2227223, 326681, 1522716, 326682, 556058, 2227227, 326685, 2227226, 326687, 1932319, 2227228, 2227229, 2227231, 326693, 326694, 326695, 326696, 3767337, 326700, 1522733, 2227244, 1522735, 2227246, 4226095, 1522738, 3259443, 3259444, 1522741, 3259445, 1293365, 3259448, 3259449, 1522746, 2227251, 1244216, 1293373, 2227258, 3259455, 3259456, 3259457, 3259458, 3259459, 3259460, 3259461, 1522751, 1522752, 1522753, 1522754, 1293385, 4996163, 1293388, 687173, 687174, 2227270, 2227276, 2227277, 1457234, 1522771, 2227278, 900181, 2227279, 2227280, 2227286, 2227289, 2227297, 3964005, 4226149, 4226151, 3554409, 4226154, 2227307, 1522796, 2227312, 4226167, 2227323, 3554428, 3554429, 2227325, 2227328, 2227331, 2227335, 2227337, 2227338, 2817161, 2227340, 4226187, 3554447, 4226191, 2227344, 2227347, 2227350, 2227351, 2227352, 1653913, 2227354, 1555611, 2227357, 2227358, 2227362, 2227365, 2227370, 4177067, 2227375, 2227377, 1653942, 2227385, 2227393, 4226243, 2227397, 2227400, 2686152, 4226249, 2227408, 4226256, 4685009, 2227413, 2686166, 4226262, 4226265, 4242652, 2227420, 4226273, 2227428, 2227429, 2227433, 4226282, 4226284, 2227441, 2227443, 2227447, 2227449, 3849473, 3849474, 4226310, 2227259, 2227465, 2227466, 2227467, 4226314, 2227260, 3849487, 2227473, 4226322, 2227480, 2227263, 2321700, 4226335, 4226337, 4226338, 2321701, 2227491, 4226341, 621862, 2227494, 2227496, 2227500, 3030318, 2227502, 4226353, 2227507, 2227508, 4226356, 4226357, 2227512, 2653497, 4226360, 3210555, 4226361, 4226362, 2227519, 3210561, 2227525, 2227526, 4226374, 4226381, 2227539, 572757, 4226389, 4226391, 4226396, 2227556, 2964840, 2227567, 2227568, 2227569, 2227571, 2227572, 2227573, 2227575, 4226425, 2227578, 2227579, 1162621, 2227581, 4849022, 2227584, 4226435, 2227592, 2227593, 475316, 2227594, 2227595, 4226443, 966030, 2227597, 2227602, 3243410, 4226450, 2227605, 2227607, 4226455, 4226458, 4226463, 2227618, 2227619, 2227623, 2227626, 1179052, 2227630, 4226478, 2227636, 1785269, 4226484, 2227638, 1621432, 1785273, 1621434, 4226491, 1637820, 2227646, 2227647, 2227649, 2227650, 1129925, 2227653, 2227655, 2227656, 2063818, 2227659, 1588684, 1179085, 1785294, 2227665, 1785298, 2227670, 2227671, 1179096, 1785305, 3572213, 4226521, 2227676, 2227677, 2227679, 146914, 3079656, 4193768, 2227690, 2227692, 2227695, 2227698, 2227699, 3243506, 2227701, 2227703, 2227709, 4750845, 4750848, 2227713, 4750849, 4750850, 4750852, 2227717, 4750853, 4750855, 4750856, 2227722, 2227723, 4226571, 2227725, 4193806, 2227727, 4193807, 4193811, 2227732, 4193812, 343578, 2227739, 2227740, 2227741, 2227745, 2227746, 4226597, 2227753, 2227754, 2227756, 4226604, 2227758, 2227760, 2227762, 2227764, 2227765, 3423800, 2227768, 2227769, 2588218, 2588220, 2227773, 2588224, 2227777, 2227778, 343621, 2227784, 2227785, 343626, 2227786, 2227789, 2227790, 343631, 1130064, 343633, 4226639, 3046992, 2227794, 2227795, 3898963, 3898969, 2227802, 2227803, 4226650, 2227805, 4226654, 4226657, 654946, 2227813, 2227817, 2227818, 2227819, 2227822, 1916527, 1916528, 2227825, 2227827, 2227828, 2227829, 2227830, 2227831, 1359484, 4832894, 2227838, 3587715, 2227843, 2227846, 2227847, 2227848, 2227849, 2227851, 2227852, 2227854, 2227855, 2227856, 2227857, 2227859, 2227860, 2227861, 2227862, 2227864, 2227865, 2227866, 2227867, 2227868, 1326749, 2227869, 1310366, 2227870, 2227871, 2227872, 2227874, 3014308, 2227880, 2227882, 2227884, 2227885, 2227888, 2227890, 2227891, 2227892, 2227896, 1507003, 1310401, 2227906, 2227912, 2375384, 2227928, 2227929, 2227936, 2227937, 1490659, 2735845, 524006, 2227943, 2227945, 982762, 245486, 2653935, 2653936, 2227952, 2653938, 2227954, 1834740, 2227956, 2227957, 2227959, 1834744, 2227961, 2227962, 2227966, 1375999, 4275968, 1834752, 2227968, 4275971, 4275972, 2227970, 4275974, 4275975, 1130248, 4275976, 4275977, 4275978, 2227972, 1834757, 1130254, 2227977, 1130256, 2227980, 2227982, 2227984, 1130260, 2227988, 1834775, 2227994, 1834780, 2998045, 2227997, 2227998, 4816668, 818978, 3587876, 4816676, 2015015, 3587880, 1834792, 2228008, 2228009, 2228010, 2228013, 2228014, 1834799, 2228018, 2228019, 818997, 818998, 2228023, 2228024, 1834809, 819000, 2228027, 819001, 819002, 2228030, 3555135, 819003, 1130305, 1130306, 1130307, 819004, 1130309, 3096390, 1392454, 1130311, 1130312, 1130314, 2228035, 1130316, 1130317, 1392460, 1392461, 1261392, 1130321, 1130322, 1130323, 1392465, 3981136, 2228045, 1392471, 819025, 2228050, 2228053, 2228055, 2228056, 1392477, 2228058, 1392479, 1392480, 2228063, 1130338, 1392482, 2228066, 1130342, 1392486, 1392487, 2228070, 2228071, 2228073, 2228075, 2228077, 2228078, 1392496, 1130354, 2228083, 1392500, 2228088, 1130366, 2228094, 2228095, 2228096, 2228100, 1130374, 2228106, 2228111, 4816789, 2228118, 2178969, 2228122, 2228128, 1130404, 2228132, 2228133, 1130407, 2260902, 2228135, 4816806, 704429, 1130413, 1130415, 2228143, 2228147, 4374452, 2228152, 4816825, 2228158, 2228159, 4816836, 3555271, 2228168, 2473929, 2228169, 2228170, 2228172, 2228174, 2228175, 2228179, 1130452, 2228180, 2228182, 1130459, 1343451, 1130461, 2228189, 2228191, 2228193, 1130466, 2228197, 2228198, 2228199, 2228203, 1130476, 2228206, 2228207, 2228209, 1130482, 2228210, 327669, 3522550, 327671, 2228216, 327673, 1130492, 2228222, 2228223]

print(len(unknowns))

Let's check if these unknows are due to new movies or new users.

In [ ]:
# Check if the number of users from the unknown indicies are all in our training data:
print('Number of users from unknown: ', len(df_test.iloc[unknowns]['userId'].unique()))
print('Number of users from unknown in training dataframe: ', 
      len(train[train['userId'].isin(list(df_test.iloc[unknowns]['userId'].unique()))]['userId'].unique()))

Okay, we don't have a problem of new users, so it must be the movies.

Let's start our Content-based approach for predicting how these users might rate these new movies.

### Content-based Filtering Rating Prediction

The pre-process:

1. Find all the movies our users of interest have rated
2. Create a dataframe of the available features of those movies and those we need to rate
3. Vectorize the features and create similarity matrix

In [ ]:
print('Number of movies of interest: ', len(train[train['userId']
                                                .isin(list(df_test.iloc[unknowns]['userId']
                                                           .unique()))]['movieId'].unique()))

print('Number of movies with extra features: ', len(df_imdb[df_imdb['movieId']
                                                    .isin(list(df_test.iloc[unknowns]['movieId']
                                                               .unique()))]['movieId'].unique()))

print('Number of movies we need to rate: ', len(df_movies[df_movies['movieId']
                                                   .isin(list(df_test.iloc[unknowns]['movieId']
                                                              .unique()))]['movieId'].unique()))
to_b_compared = set()
to_b_rated = set(list(df_test.iloc[unknowns]['movieId']))
rated = set(train[train['userId'].isin(list(df_test.iloc[unknowns]['userId'].unique()))]['movieId'])
to_b_compared |= to_b_rated
to_b_compared |= rated

print('Number of movies we can compare: ', len(to_b_compared))

In [ ]:
# Create dataframe of features 
to_b_rated_df= df_movies[df_movies['movieId'].isin(list(to_b_rated))].copy()
content_movies_df = df_movies[df_movies['movieId'].isin(list(to_b_compared))]\
                              .sample(35000, random_state=1).copy()  # Increase sample size to compare more movies, 
                                                            # decrease it to avioid RAM issues (30k needs 12GB)

content_movies_df = content_movies_df.append(to_b_rated_df, ignore_index = True)
content_movies_df = content_movies_df.drop_duplicates()
content_movies_df = pd.merge(content_movies_df, 
                             df_imdb[['movieId', 'title_cast', 
                                      'plot_keywords', 'director']], on='movieId', how='left').fillna('')
display(content_movies_df.head(3))
print(content_movies_df.shape)

In [ ]:
# Merge all features into one column
content_movies_df['features'] = (pd.Series(content_movies_df[['genres', 'title_cast', 'plot_keywords', 'director']]
                                                              .fillna('').values.tolist()).str.join('|'))
content_movies_df['features'] = content_movies_df['features'].str.strip('|')
display(content_movies_df.head())

In [ ]:
# Vectorize the words in the features column
vect = TfidfVectorizer(analyzer='word', ngram_range=(1,1),  min_df=0,
                       stop_words='english', tokenizer= (lambda x: x.split('|')), max_features=80)
                       
feature_matrix = vect.fit_transform(content_movies_df['features'].fillna(''))
print('Feature Names: ', vect.get_feature_names())

In [ ]:
# Compute similarity matrix
cosine_sim_features = cosine_similarity(feature_matrix, 
                                        feature_matrix)
print(cosine_sim_features.shape)

In [ ]:
# Convienient indexes to map between movie titles and indexes of the movies dataframe
titles = content_movies_df['title']
indices = pd.Series(content_movies_df.index, index=content_movies_df['title'])

In [ ]:
# Subset the train dataframe for the movies we going to compare
ratings = train.merge(df_movies[['movieId', 'title']], on = 'movieId',how = 'left')
display(ratings.head(2))
print(ratings.shape)
ratings_data = ratings[ratings['movieId'].isin(list(content_movies_df['movieId'].unique()))]
display(ratings_data.head(2))
print(ratings_data.shape)

### Getting the Rating

The process:

   1. Select a reference user from the database and a reference item (movie) they have _not_ rated. 
   2. For the user, gather the similarity values between the reference item and each item the user _has_ rated. 
   3. Sort the gathered similarity values in descending order. 
   4. Select the $k$ highest similarity values which are above a given threshold value, creating a collection $K$. 
   5. Compute a weighted average rating from these values, which is the sum of the similarity values of each item multiplied by its assigned user-rating, divided by the sum of the similarity values. This can be expressed in formula as:
   
   $$ \hat{R}_{ju} = \frac{\sum_{i \in K} s_{ij} \times r_{iu}}{\sum_{i \in K} s_{ij}}   $$
   
   where $\hat{R}_{ju}$ is the weighted average computed for the reference item $j$ and reference user $u$, $K$ is the collection of items, $s_{ij}$ is the similarity computed between items $i$ and $j$, and $r_{iu}$ is the known rating user $u$ has given item $i$.
   6. We return the weighted average $\hat{R}_{ju}$ as the prediction for our reference item.
   
   
We implement this algorithmic process in the function below:

In [ ]:
def content_generate_rating_estimate(movie_title, userId, rating_data, k=20, threshold=0.0):
    # Convert the book title to a numeric index for our 
    # similarity matrix
    b_idx = indices[movie_title]
    neighbors = [] # <-- Stores our collection of similarity values 
     
    # Gather the similarity ratings between each movie the user has rated
    # and the reference movie 
    for index, row in rating_data[rating_data['userId']==userId].iterrows():
        sim = cosine_sim_features[b_idx-1, indices[row['title']]-1]
        if type(sim) == np.ndarray:
            sim = sim.max()
        neighbors.append((sim, row['rating']))
    # Select the top-N values from our collection
    k_neighbors = heapq.nlargest(k, neighbors, key=lambda t: t[0])

    # Compute the weighted average using similarity scores and 
    # user item ratings. 
    simTotal, weightedSum = 0, 0
    for (simScore, rating) in k_neighbors:
        # Ensure that similarity ratings are above a given threshold
        if (simScore > threshold):
            simTotal += simScore
            weightedSum += simScore * rating
    try:
        predictedRating = weightedSum / simTotal
    except ZeroDivisionError:
        # Cold-start problem - No ratings given by user. 
        # We use the average rating for the reference item as a proxy in this case 
        predictedRating = np.mean(rating_data[rating_data['title']==movie_title]['rating'])
    return predictedRating

We'll test our model on some data we have ratings for first and valdate it, then we'll run the model on new test data.

In [ ]:
# Randomly select samples to test.
ratings_data.sample(10)

In [ ]:
%%time
title = 'Men in Black (a.k.a. MIB) (1997)'
actual_rating = ratings_data[(ratings_data['userId'] == 108943) & (ratings_data['title'] == title)]['rating'].values[0]
pred_rating = content_generate_rating_estimate(movie_title=title, userId=108943, rating_data=ratings_data)
print (f"Title - {title}")
print ("---")
print (f"Actual rating: \t\t {actual_rating}")
print (f"Predicted rating: \t {pred_rating}")

In [ ]:
%%time
title = 'Shaft (2000)'
actual_rating = ratings_data[(ratings_data['userId'] == 84758) & (ratings_data['title'] == title)]['rating'].values[0]
pred_rating = content_generate_rating_estimate(movie_title=title, userId=84758, rating_data=ratings_data)
print (f"Title - {title}")
print ("---")
print (f"Actual rating: \t\t {actual_rating}")
print (f"Predicted rating: \t {pred_rating}")

### The Heterogeneous Approach to Hybrid Recommendation

In [ ]:
# Subset our test dataframe for rows deom the unknown indices
test = df_test.merge(df_movies[['movieId', 'title']], on = 'movieId', how='left')
test_for_content_based = test.iloc[unknowns].copy()
display(test_for_content_based.head())
print(test_for_content_based.shape)

Now we have to predict on the above dataframe and merge the predictions on the index column with that of the predictions from our best collaborative model. 
Becuase we are merging predictions and not aggregating the predictions by find  finding the mean, we call this method Heterogeneous, rather than homogeneous.

We should end with a dataframe of predictions, where 6451 of those predictions comes from the content based model above, and the rest comes from our best collaborative model. 

#### Note:    

#### After multiple attempts to predict on the full 6451 rows above, we have not been able to find a feasible method that results in a reasonable prediction exectution time. The prediction function call runs for 6+ hours and we were not able to find datatypes or methods that decrease this time and give us a good amount of predictions to use. Our algorithm will need more work.

#### We therefore subset the dataframe for n rows and predict only on that. 

In [ ]:
%%time
n = 10
ratings_predictions = [content_generate_rating_estimate(row.title, row.userId, ratings_data) 
                       for _, row in test_for_content_based[:n].iterrows()]

Here we can see it took 1 min  to predict 10 rows.

If we calculate it, we have 6383 to predict on, meaning the execution time will be 638.3 minutes or 638.3/60 = **10.6 Hours !!**

In [ ]:
subset_of_test_content = test_for_content_based[:n]
subset_of_test_content['rating'] = ratings_predictions

In [ ]:
subset_of_test_content

This is where we will stop for now and consider prebuilt hybrid algorithms that use a Homogeneous Hybrid Approach.

### The Homogeneuos Approach to Hybrid Recommendation

The LibRecommender package has a few recommendation models that allows you to pass it data with feature and rating columns. These algorithms will use content-based-filtering and and collaborative model-based filtering to aggregate the prediction of each fitering type and return a single prediction. 

We need to prepare the data for our librecommender model. Here we use the genre_encode and cast encode dataframe, merge it to our train dataframe and pass it to the final preprocessing stage done by the librecommender dataset functions, which will return a train, test and validation split for the models to use.

In [ ]:
%%time
""" Creating more features from genre and cast"""

mergeded_train = df_train.copy()
mergeded_train = pd.merge(mergeded_train, genre_encode,
                          on='movieId', how='left')
mergeded_train = pd.merge(mergeded_train, cast_encode,
                          on='movieId', how='left')

In [ ]:
gc.collect()
del genre_encode, cast_encode
sleep(10)

In [ ]:
%%time
mergeded_train = pd.merge(mergeded_train, df_imdb[['movieId', 'runtime',
                                                   'budget']], on='movieId').fillna(0)
display(mergeded_train.head())
print(mergeded_train.shape)

In [ ]:
columns = list(mergeded_train.columns)
print(columns[4:])

In [ ]:
%%time
#mergeded_train = pd.read_csv('./train_feat.csv')
mergeded_train.rename(columns={'userId': 'user', 'movieId': 'item', 'rating': 'label',
                               'timestamp': 'time'}, inplace=True)

columns = list(mergeded_train.columns)
# We have to specify which columns are the content based features
sparse_col = columns[4:]
item_col = columns[4:]

# split whole data into three folds for training, evaluating and testing
train_data, eval_data, test_data = random_split(
    mergeded_train, multi_ratios=[0.99, 0.005, 0.005])


train_data, data_info = DatasetFeat.build_trainset(train_data, item_col=item_col,
                                                   sparse_col=sparse_col)
eval_data = DatasetFeat.build_evalset(eval_data)
test_data = DatasetFeat.build_testset(test_data)
print(data_info)

In [ ]:
gc.collect()
del mergeded_train, columns, sparse_col, item_col
sleep(10)

### The Hybrid Models

### FM

Factorization Machines (FM) which are a new model class that combines the advantages of Support Vector Machines (SVM) with factorization models. Like SVMs, FMs are a general predictor working with any real valued feature vector.

In [ ]:
%%time
reset_state("FM")
fm = FM("rating", data_info, embed_size=16, n_epochs=5,
        lr=0.005, lr_decay=True, reg=0.000001, batch_size=10000,
        num_neg=1, use_bn=True, dropout_rate=None, tf_sess_config=None)
fm.fit(train_data, verbose=2, shuffle=True, eval_data=eval_data,
       metrics=["rmse", "mae", "r2"])

# do final evaluation on test data
print("evaluate_result: ", evaluate(model=fm, data=test_data,
                                    metrics=["rmse", "mae"]))

The FM model took 3.5 min and 5 epochs to get a test RMSE  of 0.8083.
Now let's see it's predictions.

In [ ]:
prediction = fm.predict(user=5163, item=57669)
recommendation = fm.recommend_user(user=5163, n_rec=5)
print("prediction: ", prediction)

rec = []
ratings = []
recommended_movies = df_movies.copy()

for r in recommendation:
    rec.append(r[0])
    ratings.append(r[1])
    
recommended_movies = recommended_movies[recommended_movies['movieId'].isin(rec)].set_index('movieId').copy()
recommended_movies['rating'] = ratings

print('User 5163 will be recommended:')
display(recommended_movies.sort_values('rating', ascending =False))

In [ ]:
gc.collect()
del fm
sleep(10)

### Deep FM

DeepFM: A Factorization-Machine based Neural Network for CTR Prediction. The proposed model, DeepFM, combines the power of factorization machines for recommendation and deep learning for feature learning in a new neural network architecture

In [ ]:
%%time
reset_state("DeepFM")
deepfm = DeepFM("rating", data_info, embed_size=16, n_epochs=3,
                lr=5e-3, lr_decay=True, reg=0.000001, batch_size=10000,
                num_neg=1, use_bn=False, dropout_rate=None,
                hidden_units="128,64,32,16,8,4", tf_sess_config=None)
deepfm.fit(train_data, verbose=2, shuffle=True, eval_data=eval_data,
           metrics=["rmse", "mae", "r2"])

print("evaluate_result: ", evaluate(model=deepfm, data=test_data,
                                    metrics=["rmse", "mae"]))

The DeepFM took 3min and 3 epochs to get a test RMSE  of 0.8094.

In [ ]:
prediction = fm.predict(user=5163, item=57669)
recommendation = fm.recommend_user(user=5163, n_rec=5)
print("prediction: ", prediction)

rec = []
ratings = []
recommended_movies = df_movies.copy()

for r in recommendation:
    rec.append(r[0])
    ratings.append(r[1])
    
recommended_movies = recommended_movies[recommended_movies['movieId'].isin(rec)].set_index('movieId').copy()
recommended_movies['rating'] = ratings

print('User 5163 will be recommended:')
display(recommended_movies.sort_values('rating', ascending =False))

In [ ]:
gc.collect()
del deepfm
sleep(10)

# The Recommender Engine 

Now that we have trained, tested and validated a few recommendation models, we can now go ahead and build a recommender engine. 
A **Collaborative model-based Reccomender Engine** to be precise. 

The Reccomender Engine will take in a list of movies the user likes and will return a list of movie reccomendations based on the provided list.

In [ ]:
%%time
data = train.copy()
data.rename(columns={'userId': 'user', 'movieId': 'item', 'rating': 'label',
                               'timestamp': 'time'}, inplace=True)
train_data, data_info = DatasetPure.build_trainset(data)

reset_state('libSVD')
svd_lib = libSVD(task="rating", data_info=data_info, embed_size=16, n_epochs=5, lr=0.004, batch_size=9000,
                 batch_sampling=True, reg=0.000001, num_neg=-5)
svd_lib.fit(train_data, verbose=2)

In [ ]:
ratings_df = train.copy()
ratings_df.drop(['timestamp'], axis=1, inplace=True)

ratings = pd.merge(ratings_df, df_movies[['movieId', 'title']], on='movieId')


def prediction_item(item_id):
    """Map a given favourite movie to users within the
       MovieLens dataset with the same preference.

    Parameters
    ----------
    item_id : int
        A MovieLens Movie ID.

    Returns
    -------
    list
        User IDs of users with similar high ratings for the given movie.

    """
    # Using Surprise Model
    if str(model)[1:8] != 'libreco':
        # Data preprosessing
        reader = Reader(rating_scale=(0.5, 5))
        load_df = Dataset.load_from_df(ratings_df, reader)
        a_train = load_df.build_full_trainset()

        predictions = []
        for ui in a_train.all_users():
            predictions.append(model.predict(
                iid=item_id, uid=ui, verbose=False))

    # Using LibReco Model
    else:
        pred = list(svd_lib.predict(
            user=train['userId'].unique(), item=item_id))
        users = list(train['userId'])
        predictions = [(i, j) for i, j in zip(pred, users)]

    return predictions


def get_user_movies(df, user_list):
    """
    :param df:
    :param user_list:
    :return: dataframe subset of train data
    """
    temp = pd.DataFrame()
    for i in user_list:
        temp_df = df[df['userId'] == i]
        temp = temp.append(temp_df)
    return temp


def pred_movies(movie_list):
    """Maps the given favourite movies selected within the app to corresponding
    users within the MovieLens dataset.

    Parameters
    ----------
    movie_list : list
        Three favourite movies selected by the app user.

    Returns
    -------
    list
        User-ID's of users with similar high ratings for each movie.

    """
    # Store the id of users
    id_store = []
    # For each movie selected by a user of the app,
    # predict a corresponding user within the dataset with the highest rating

    # Using Surprise Model
    if str(model)[1:8] != 'libreco':
        for i in movie_list:
            movieID = df_movies[df_movies['title'] == i]['movieId'].values[0]
            predictions = prediction_item(item_id=movieID)
            predictions.sort(key=lambda x: x.est, reverse=True)
            # Take the top 10 user id's from each movie with highest rankings
            for pred in predictions[:10]:
                id_store.append(pred.uid)

    # Using LibReco Model
    else:
        for i in movie_list:
            movieID = df_movies[df_movies['title'] == i]['movieId'].values[0]
            predictions = prediction_item(item_id=movieID)
            predictions.sort(reverse=True)
            # Take the top 10 user id's from each movie with highest rankings
            for i, j in predictions[:10]:
                id_store.append(j)

    # Return a list of user id's
    return id_store


def recommend_movies(movie_list, top_n=10):
    """Performs Collaborative filtering based upon a list of movies supplied
       by the app user.

    Parameters
    ----------
    movie_list : list (str)
        Favorite movies chosen by the app user.
    top_n : type
        Number of top recommendations to return to the user.

    Returns
    -------
    list (str)
        Titles of the top-n movie recommendations to the user.

    """
    user_ids = pred_movies(movie_list)

    temp = get_user_movies(ratings, user_ids)

    movie_ids = []
    for i in movie_list:
        movieID = df_movies[df_movies['title'] == i]['movieId'].values[0]
        movie_ids.append(movieID)

    # Add new user with ratings to userlist
    new_row1 = {'userId': 1000000,
                'movieId': movie_ids[0], 'rating': 5.0, 'title': movie_list[0]}
    new_row2 = {'userId': 1000000,
                'movieId': movie_ids[1], 'rating': 5.0, 'title': movie_list[1]}
    new_row3 = {'userId': 1000000,
                'movieId': movie_ids[2], 'rating': 5.0, 'title': movie_list[2]}
    temp = temp.append([new_row1, new_row2, new_row3], ignore_index=True)

    user_ratings = temp.pivot_table(
        index='userId', columns='title', values='rating').fillna(0)
    item_similarity_df = user_ratings.corr(method='pearson')

    def get_similar_movies(movie_name, user_rating=5):
        similar_score = item_similarity_df[movie_name] * user_rating
        similar_score = similar_score.sort_values(ascending=False)
        return similar_score

    similar_movies = pd.DataFrame()

    for movie in movie_list:
        similar_movies = similar_movies.append(
            get_similar_movies(movie, 5), ignore_index=True)

    recommended_movies = []
    for i in similar_movies.sum().sort_values(ascending=False).index:
        if i in movie_list:
            pass
        else:
            recommended_movies.append(i)

    return recommended_movies[:10]

In [ ]:
movie_list = ['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)']
model = svd_lib
recommend_movies(movie_list)


## 9) Creating a Kaggle Submission


In [ ]:
def create_submission(algo, save_name):

    ratings_predictions = [algo.predict(
        row.userId, row.movieId) for _, row in df_test.iterrows()]
    df_pred = pd.DataFrame(ratings_predictions)
    df_pred = df_pred.rename(
        columns={'uid': 'userId', 'iid': 'movieId', 'est': 'rating'})
    df_pred.drop(['r_ui', 'details'], axis=1, inplace=True)
    # Create ID column
    df_pred['Id'] = df_pred.apply(
        lambda x: '%s_%s' % (x['userId'], x['movieId']), axis=1)
    df_pred['Id'] = df_pred.apply(
        lambda x: '%s_%s' % (x['userId'], x['movieId']), axis=1)
    df_pred = df_pred[['Id', 'rating']]
    df_pred.to_csv(save_name + '.csv', index=False)

    return pd.read_csv('./' + save_name + '.csv')


def create_librec_submission(svdpp, save_name):
    svdpp_pred = svdpp.predict(
        df_test['userId'], df_test['movieId'], cold_start="popular")
    df_pred = df_test.copy()
    df_pred['rating'] = svdpp_pred
    df_pred['Id'] = df_pred.apply(
        lambda x: '%s_%s' % (x['userId'], x['movieId']), axis=1)
    df_pred['Id'] = df_pred.apply(
        lambda x: '%s_%s' % (x['userId'], x['movieId']), axis=1)
    df_pred = df_pred[['Id', 'rating']]
    df_pred.to_csv(save_name + '.csv', index=False)
    return pd.read_csv('./' + save_name + '.csv')

#### Train suprise SVD model on full trainset and create submission

In [ ]:
%%time
"""
ratings = df_train.copy()
ratings_dict = {'itemID': list(ratings.movieId),
                'userID': list(ratings.userId),
                'rating': list(ratings.rating)}

df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0.5, 5.0))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)


trainset = data.build_full_trainset()
algo = SVD(random_state = 1, n_epochs= 16, verbose=True, n_factors =220, 
           lr_all = 0.016, reg_all =0.02, init_std_dev = 0.005, init_mean = -0.037)
algo.fit(trainset)

"""

In [ ]:
%%time
"""
submission = create_submission(algo, 'SVD_tunedA3')
submission.head(10)

"""

## 10) Model Pickling

In [ ]:
# Pickle The Model
"""
model_save_path = "SVD_tunedA3.pkl"
with open(model_save_path, 'wb') as file:
    pickle.dump(algo, file)
    
"""

In [ ]:
# Unpickle
"""
model_load_path = "/content/drive/MyDrive/Unsupervised Models/SVD_tunedA3.pkl"
with open(model_load_path,'rb') as file:
    algo = pickle.load(file)
    
"""

#### Train LibRecommender SVDpp model on full trainset and create submission

In [ ]:
%%time
"""
data = train.copy()
data.rename(columns={'userId': 'user', 'movieId': 'item', 'rating': 'label',
                               'timestamp': 'time'}, inplace=True)

train_data, data_info = DatasetPure.build_trainset(data)

reset_state('SVDpp')
svdpp = SVDpp(task="rating", data_info=data_info, embed_size=16, reg=0.000001, num_neg=-5,
              n_epochs=8, lr=0.0048, batch_size=10000, batch_sampling=True)
              
svdpp.fit(train_data, verbose=2)

"""

In [ ]:
%%time
"""
librec_submission = create_librec_submission(svdpp, 'SVDpp_librecB4')
librec_submission.head(10)

"""

Save LibRec SVDpp model

In [ ]:
"""
data_info.save(path="SVDpp_model_path")
svdpp.save(path="SVDpp_model_path", model_name="svdpp_model", manual=True,
                inference_only=False)
                
"""

In [ ]:
"""
tf.compat.v1.reset_default_graph()
data_info = DataInfo.load("SVDpp_model_path")

svdpp_loaded = SVDpp(task="rating", data_info=data_info, embed_size=16, n_epochs=2, 
                                                lr=0.005,batch_size=10000, batch_sampling=True)

data = pd.read_csv(root_path + 'train.csv', header=0, names = ["user", "item", "label", "time"])
train_data, eval_data, test_data = random_split(data, multi_ratios=[0.99, 0.005, 0.005])
train_data, data_info = DatasetPure.build_trainset(train_data)
svdpp_loaded.rebuild_graph(path="SVDpp_model_path", model_name="svdpp_model",
                        full_assign=True, train_data=train_data)

svdpp_loaded.predict(df_train['userId'][:10], df_train['movieId'][:10])

"""

## 11) Conclusion

watch me:  
Enable Group Reccomendations.

0. Start Fresh: It means be intentional about the movies you choose to watch. 
1. Start a group, share the userID.
2. Login with the userID (The first sucessful login requires at least 2 users to acknowledge they are using the UserID).
3. Whoever logs in first will have zero movies watched/rated, but the opertunity to choose movie catorgories
    they and their friends might be interested in.
   ()
4. Whoever logs in next with the userID will be recommended movies from those the previous user has rated/watched.
    This user will also be given a opportunity to choose movie catorgories they are interested in. 
5. Repeat step 4 for all new users who are given the userID. 
6. All the ratings and watched-movies data will be stored under the single UserID, thereby allowing our recommender 
    algorithm to reccomend movies it predicts all the users of the userID might enjoy.
7. You will end up exploring new movies from a user you chose might be reasonably similar to you. Collaborative based.
8. You also end up exloring new content and based on how you rate it and how the other users rate it, you might 
   be recommended similar content.

This platform is for being intentional about watching movies, it is a new account you chose to share with someone/ some friends.
It makes movie night easier when it comes around, it also makes movie night anytime if all users are logged in at the same time.
The usecase of this patform grows the more you think about it.
It can be shared by school goers and monitored by teachers who distribute the userIDs, allowing groups of students access to movies
on school content and recieve reccomendations on movies similar to that content.

Extra features can be an online chat while browsing among other things.

This group recommender system can easily be deployed as a service to esabished online movie streaming platforms such as
Netflix, Amazon Prime Video, Hulu, Disney+ and HBO Max.
 
We already a collaborative model that can predict given just a user Id and the number of records. 
We also have a content based model that can predict similar movies given a list of titles. 
All that needs to be done regarding the coding of the demo is  ' if '  statements.